# Prompt模板

这个模板有小改动, 和生成这些code的不是完全一致. 

In [ ]:
prompt_meu_coding_bse_08 = """
# 身份
你是一个精通法律的程序员. 擅长编写代码以检验特定案例对于特定法律要求的合规性. 

# 任务
你现在面临一个编写代码以检验某法律事件的合规性的任务, 你需要编写一个函数, 这个函数的输入是一个日频的pandas dataframe, 每一行是在某个交易日内公司的情况和股东的情况, 这个函数会检验该dataframe的每一行在某一个给定的法律最小可执行单元上的合规性, 标注在特定的列上, 并返回标注后的dataframe. 

# 更多要求
 - 你可以进行任意长度的思考, 然后用<CODE></CODE>包裹你的代码. 
 - 你需要按照下面这个框架编写代码: 1. 标记valid的subject(meu_n_k_subject); 2. 标记valid的condition(meu_n_k_condition); 3. 标记valid的constrain(meu_n_k_constrain); 4. 在dataframe中记录subject, condition和constrain的valid情况; 5. 计算整体的违约情况并标记在dataframe上. 

# example
下面是一个例子:

## 输入
法律的最小可执行单元编号: MEU_4_1
法律的最小可执行单元内容: {{"subject": "上市公司大股东 | 董监高", "condition": "计划通过本所集中竞价或大宗交易减持股份", "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", "contextual_info": NaN}}
必要的额外信息: NaN

## 输出
<CODE>
import pandas a pd

def check_meu_4_1(df):
    '''
    代码的注释要清晰, 展示你分别验证subject, condition和constrain的过程.
    "subject": "上市公司大股东 | 董监高", 
    "condition": "计划通过本所集中竞价或大宗交易减持股份", 
    "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", 
    "contextual_info": NaN
    '''

    df['meu_4_2_subject'] = False  # 初始化subject列
    df['meu_4_2_condition'] = False  # 初始化condition列
    df['meu_4_2_constrain'] = False  # 初始化constrain列
    
    # 标记valid的subject
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    is_director = df['股东身份'].isin(['董监高'])
    valid_subject = is_major_shareholder | is_director

    # 标记valid的condition
    valid_condition = df['减持方式'].isin(['竞价交易', '大宗交易'])

    # 标记valid的constrain
    df['交易日差'] = (df['计划开始日'] - df['计划披露日']).dt.days
    has_plan = df['存在减持计划']
    valid_constraint = df['交易日差'] >= 15 & df['存在减持计划']
    
    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_4_1_subject'] = True
    df.loc[valid_condition, 'meu_4_1_condition'] = True
    df.loc[valid_constraint, 'meu_4_1_constraint'] = True
    
    # 判断违规情况：满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )
    
    # 标记违规行
    df.loc[is_violation, 'meu_4_1_compliance'] = False
    
    return df

</CODE>
    
# 下面轮到你来完成这个任务
法律的最小可执行单元编号: {meu_id}
法律的最小可执行单元内容: {meu_content}
必要的额外信息: {note}


# 附加信息
## 你可以操作的dataframe的columns: 
['日期', '日收益率', '收盘价', '前收盘价', '上市日期', '发行价格', '净资产增长率', '净利润增长率', '净资产',
'净利润', '总股本', '每股净资产', '每股净利润', '公告类型', '公告日期', '复权因子', '收盘价减每股净资产',
'收盘价减每股净利润', '公司涉嫌证券期货违法犯罪事件', '持股比例', '持股数量', '股东身份', '减持方式', '拟减持原因',
'存在减持计划', '计划减持比例', '计划价格下限', '计划价格上限', '当日减持比例', '累计减持比例', '计划披露日',
'计划开始日', '计划结束日', '离任日期', '公司简称', '股东', '股东涉嫌证券期货违法犯罪事件']

## 其中: 
- 所有日期相关的列都已经是pd.datetime格式, ['日期']列本身就是交易日, 不需要额外计算交易日, 但是自然日需要计算. 
- 所有价格的单位都是元而非万元, 千元或百万元; 所有的比例都是小数而非百分数
- 这是一个二维的面板数据, 根据['日期', '公司简称', '股东']可以定位到特定公司的特定股东在特定交易日的状态, 以及公司在该日的状态. 如果你要分公司和股东地考虑一些合规要求, 需要考虑按照 ['公司简称', '股东']分组并在组内完成. 
- 交易方式有['竞价交易', '大宗交易', '融券卖出'], 竞价交易包括集中竞价. 
- ['公司/股东涉嫌证券期货违法犯罪事件'] 有['被中国证监会及其派出机构立案调查', '中国证监会及其派出机构立案调查结束', '行政处罚决定作出', '被司法机关立案侦查', '司法机关立案侦查结束', '被本所公开谴责']
- ['股东身份'] 有 ['董监高', '离任董监高', '控股股东', '实际控制人', '持股5%以上股东', '普通股东'], 这些标签不会并列出现, 因此有时候董监高/离任董监高持股在5%以上但并没有标明'持股5%以上股东', 你需要从['持股比例']手动筛选. 
- ['存在减持计划']是一个布尔值，表示该股东当天是否有减持计划
- 假设公司上市发行价就是最早的非空的收盘价, 公司发行时的控股股东和实际控制人就是第一个非空的收盘价的同一天的控股股东和实际控制人. 
- ['持股比例'] 全部是公开上市前取得的股份. 不考虑承诺和限制出售的股份. 

"""

# Law Article 4

## MEU_4_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 | 董监高 |
| condition | 计划通过本所集中竞价或大宗交易减持股份 |
| constraint | 应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1432 |
| completion_tokens | 741 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_4_1(df):
    '''
    检查减持计划披露合规性
    "subject": "上市公司大股东 | 董监高", 
    "condition": "计划通过本所集中竞价或大宗交易减持股份", 
    "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划"
    
    处理逻辑:
    1. 识别主体: 大股东(持股≥5%)或董监高
    2. 识别条件: 计划通过集中竞价或大宗交易减持
    3. 检查约束: 提前15个交易日披露减持计划
    4. 标记合规情况
    '''
    
    # 初始化结果列
    df['meu_4_1_subject'] = False
    df['meu_4_1_condition'] = False
    df['meu_4_1_constraint'] = False
    df['meu_4_1_compliance'] = True  # 默认合规
    
    # 1. 验证subject条件: 大股东或董监高
    # 大股东条件: 持股比例≥5%或身份为控股股东/实际控制人/持股5%以上股东
    is_major_shareholder = (
        (df['持股比例'] >= 0.05) |
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东'])
    )
    # 董监高条件: 身份为董监高(不包括离任)
    is_director = df['股东身份'] == '董监高'
    valid_subject = is_major_shareholder | is_director
    
    # 2. 验证condition条件: 计划通过集中竞价或大宗交易减持
    valid_condition = (
        df['存在减持计划'] & 
        df['减持方式'].isin(['竞价交易', '大宗交易'])
    )
    
    # 3. 验证constraint条件: 提前15个交易日披露
    # 计算计划开始日与披露日的交易日差
    df['交易日差'] = (df['计划开始日'] - df['计划披露日']).dt.days
    valid_constraint = df['交易日差'] >= 15
    
    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_4_1_subject'] = True
    df.loc[valid_condition, 'meu_4_1_condition'] = True
    df.loc[valid_constraint, 'meu_4_1_constraint'] = True
    
    # 识别违规情况: 满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )
    df.loc[is_violation, 'meu_4_1_compliance'] = False
    
    # 对于没有减持计划的记录，标记为不适用
    no_plan = ~df['存在减持计划']
    df.loc[no_plan, ['meu_4_1_subject', 'meu_4_1_condition', 'meu_4_1_constraint']] = pd.NA
    df.loc[no_plan, 'meu_4_1_compliance'] = pd.NA
    
    return df

---

## MEU_4_7


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 | 董监高 |
| condition | 披露减持计划 |
| constraint | 每次披露的减持计划中减持时间区间不得超过3个月 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1414 |
| completion_tokens | 620 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_4_7(df):
    '''
    检查减持计划披露的时间区间是否符合不超过3个月的规定
    
    "subject": "上市公司大股东 | 董监高", 
    "condition": "披露减持计划", 
    "constraint": "每次披露的减持计划中减持时间区间不得超过3个月", 
    "contextual_info": NaN
    
    实现逻辑:
    1. 首先识别符合subject要求的股东(大股东或董监高)
    2. 然后检查是否存在减持计划披露(condition)
    3. 最后验证减持计划的时间区间是否<=3个月(constraint)
    '''
    
    # 初始化结果列
    df['meu_4_7_subject'] = False
    df['meu_4_7_condition'] = False
    df['meu_4_7_constraint'] = False
    df['meu_4_7_compliance'] = True  # 默认合规
    
    # 1. 验证subject: 大股东或董监高
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    is_director = df['股东身份'].isin(['董监高', '离任董监高'])
    valid_subject = is_major_shareholder | is_director
    
    # 2. 验证condition: 披露减持计划
    valid_condition = df['存在减持计划']
    
    # 3. 验证constraint: 减持时间区间<=3个月
    # 计算减持计划的时间区间(自然日)
    df['减持区间天数'] = (df['计划结束日'] - df['计划开始日']).dt.days
    valid_constraint = df['减持区间天数'] <= 90  # 3个月按90天计算
    
    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_4_7_subject'] = True
    df.loc[valid_condition, 'meu_4_7_condition'] = True
    df.loc[valid_constraint, 'meu_4_7_constraint'] = True
    
    # 判断违规情况: 满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )
    
    # 标记违规行
    df.loc[is_violation, 'meu_4_7_compliance'] = False
    
    # 添加违规原因说明
    df.loc[is_violation, 'meu_4_7_violation_reason'] = '减持计划时间区间超过3个月'
    
    return df

---

## MEU_4_8


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 | 董监高 |
| condition | 拟在3个月内通过集中竞价交易减持股份的总数超过公司股份总数1% |
| constraint | 应当在首次卖出的30个交易日前预先披露减持计划 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1429 |
| completion_tokens | 897 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_4_8(df):
    '''
    检查MEU_4_8合规性:
    "subject": "上市公司大股东 | 董监高", 
    "condition": "拟在3个月内通过集中竞价交易减持股份的总数超过公司股份总数1%", 
    "constraint": "应当在首次卖出的30个交易日前预先披露减持计划", 
    "contextual_info": NaN
    
    实现逻辑:
    1. 首先验证subject是否符合要求(大股东或董监高)
    2. 然后验证condition是否符合(3个月内通过集中竞价减持超过1%)
    3. 最后验证constraint是否符合(提前30个交易日披露)
    4. 标记各条件满足情况并判断是否违规
    '''
    
    # 初始化结果列
    df['meu_4_8_subject'] = False
    df['meu_4_8_condition'] = False
    df['meu_4_8_constraint'] = False
    df['meu_4_8_compliance'] = True  # 默认合规
    
    # 1. 验证subject条件 (大股东或董监高)
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    is_director = df['股东身份'].isin(['董监高'])
    valid_subject = is_major_shareholder | is_director
    df.loc[valid_subject, 'meu_4_8_subject'] = True
    
    # 2. 验证condition条件 (3个月内通过集中竞价减持超过1%)
    # 需要按公司和股东分组计算未来3个月(约63个交易日)的计划减持比例
    df_grouped = df.groupby(['公司简称', '股东'])
    
    # 计算每个减持计划在未来3个月内的累计减持比例
    df['未来3个月累计减持比例'] = 0.0
    
    for name, group in df_grouped:
        # 对每个股东在公司中的记录按日期排序
        group = group.sort_values('日期')
        
        # 计算每个日期后63个交易日的窗口
        for idx, row in group.iterrows():
            if not row['存在减持计划']:
                continue
                
            start_date = row['日期']
            end_date = start_date + pd.Timedelta(days=90)  # 近似3个月
            
            # 计算该计划在未来3个月的累计减持比例
            mask = (
                (group['日期'] >= start_date) & 
                (group['日期'] <= end_date) &
                (group['减持方式'] == '竞价交易')
            )
            total_reduction = group.loc[mask, '计划减持比例'].sum()
            
            df.loc[idx, '未来3个月累计减持比例'] = total_reduction
    
    # 条件判断: 存在减持计划且未来3个月集中竞价减持超过1%
    valid_condition = (
        df['存在减持计划'] & 
        (df['减持方式'] == '竞价交易') & 
        (df['未来3个月累计减持比例'] > 0.01)
    )
    df.loc[valid_condition, 'meu_4_8_condition'] = True
    
    # 3. 验证constraint条件 (提前30个交易日披露)
    # 计算计划披露日与计划开始日之间的交易日差
    df['交易日差'] = (df['计划开始日'] - df['计划披露日']).dt.days
    valid_constraint = df['交易日差'] >= 30
    df.loc[valid_constraint, 'meu_4_8_constraint'] = True
    
    # 判断违规情况: 满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )
    df.loc[is_violation, 'meu_4_8_compliance'] = False
    
    return df

---

# Law Article 9

## MEU_9_1


| 字段 | 内容 |
|------|------|
| subject | 控股股东 | 实际控制人 | 董监高 |
| condition | 公司上市时未盈利且处于实现盈利前阶段 |
| constraint | 自公司股票上市之日起2个完整会计年度内不得减持公开发行并上市前股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1433 |
| completion_tokens | 994 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_9_1(df):
    '''
    检查控股股东、实际控制人或董监高在公司上市时未盈利且处于实现盈利前阶段时，
    自公司股票上市之日起2个完整会计年度内不得减持公开发行并上市前股份的合规性
    
    "subject": "控股股东 | 实际控制人 | 董监高", 
    "condition": "公司上市时未盈利且处于实现盈利前阶段", 
    "constraint": "自公司股票上市之日起2个完整会计年度内不得减持公开发行并上市前股份", 
    "contextual_info": NaN
    '''
    
    # 初始化结果列
    df['meu_9_1_subject'] = False
    df['meu_9_1_condition'] = False
    df['meu_9_1_constraint'] = False
    df['meu_9_1_compliance'] = True  # 默认合规
    
    # 1. 验证subject条件：控股股东、实际控制人或董监高
    is_controlling = df['股东身份'].isin(['控股股东', '实际控制人'])
    is_director = df['股东身份'].isin(['董监高'])
    valid_subject = is_controlling | is_director
    
    # 2. 验证condition条件：公司上市时未盈利且处于实现盈利前阶段
    # 2.1 找出每个公司的上市日期和发行价
    company_info = df.groupby('公司简称').agg({
        '上市日期': 'first',
        '每股净利润': 'first'
    }).reset_index()
    company_info.rename(columns={'每股净利润': '上市时每股净利润'}, inplace=True)
    
    # 2.2 合并回原DataFrame
    df = pd.merge(df, company_info, on='公司简称', how='left')
    
    # 2.3 判断公司上市时是否未盈利（每股净利润<=0）
    unprofitable_at_ipo = df['上市时每股净利润'] <= 0
    
    # 2.4 判断当前是否仍处于实现盈利前阶段（当前每股净利润<=0）
    still_unprofitable = df['每股净利润'] <= 0
    
    valid_condition = unprofitable_at_ipo & still_unprofitable
    
    # 3. 验证constraint条件：自上市之日起2个完整会计年度内不得减持
    # 3.1 计算当前日期与上市日期的年份差
    df['上市年限'] = (df['日期'].dt.year - df['上市日期'].dt.year)
    
    # 3.2 考虑完整会计年度（需要跨过12月31日）
    # 如果当前月份已经超过上市月份，则已满完整年度
    full_years_passed = (df['日期'].dt.year - df['上市日期'].dt.year) 
    full_years_passed = full_years_passed - ((df['日期'].dt.month < df['上市日期'].dt.month) | 
                                           ((df['日期'].dt.month == df['上市日期'].dt.month) & 
                                            (df['日期'].dt.day < df['上市日期'].dt.day)))
    
    # 3.3 判断是否在2个完整会计年度内
    within_2_years = full_years_passed < 2
    
    # 3.4 判断是否有减持行为（当日减持比例>0）
    has_reduction = df['当日减持比例'] > 0
    
    # 约束条件：如果在2年内减持了股份，则违反约束
    valid_constraint = ~(within_2_years & has_reduction)
    
    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_9_1_subject'] = True
    df.loc[valid_condition, 'meu_9_1_condition'] = True
    df.loc[valid_constraint, 'meu_9_1_constraint'] = True
    
    # 判断违规情况：满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )
    
    # 标记违规行
    df.loc[is_violation, 'meu_9_1_compliance'] = False
    
    # 清理临时列
    df.drop(['上市时每股净利润', '上市年限'], axis=1, inplace=True, errors='ignore')
    
    return df

---

## MEU_9_3


| 字段 | 内容 |
|------|------|
| subject | 董监高 |
| condition | 在前款规定的2个完整会计年度期间内离职 |
| constraint | 应当继续遵守前款规定的减持限制 |
| contextual_info | nan |
| note | nan |
| relation | refer_to |
| target | MEU_9_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1413 |
| completion_tokens | 828 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_9_3(df):
    '''
    检查MEU_9_3合规性:
    "subject": "董监高", 
    "condition": "在前款规定的2个完整会计年度期间内离职", 
    "constraint": "应当继续遵守前款规定的减持限制", 
    "contextual_info": NaN
    
    实现逻辑:
    1. subject: 检查是否为董监高或离任董监高
    2. condition: 检查是否在离职后2个完整会计年度内
    3. constraint: 检查是否遵守减持限制(假设前款规定的减持限制是指董监高减持限制)
    '''
    
    # 初始化结果列
    df['meu_9_3_subject'] = False
    df['meu_9_3_condition'] = False
    df['meu_9_3_constraint'] = False
    df['meu_9_3_compliance'] = True  # 默认合规
    
    # 1. 检查subject条件 - 董监高或离任董监高
    is_director = df['股东身份'].isin(['董监高', '离任董监高'])
    df.loc[is_director, 'meu_9_3_subject'] = True
    
    # 2. 检查condition条件 - 在离职后2个完整会计年度内
    # 获取每个公司的会计年度信息(假设会计年度是自然年)
    df['会计年度'] = df['日期'].dt.year
    
    # 对每个股东计算离职日期和2个完整会计年度后的截止日期
    grouped = df.groupby(['公司简称', '股东'])
    
    for (company, shareholder), group in grouped:
        # 获取该股东的离任日期(如果有)
        leave_dates = group['离任日期'].dropna()
        if not leave_dates.empty:
            leave_date = leave_dates.iloc[0]  # 取第一个离任日期
            leave_year = leave_date.year
            
            # 计算2个完整会计年度后的截止日期(离职当年+2年)
            cutoff_date = pd.Timestamp(year=leave_year + 2, month=12, day=31)
            
            # 标记在离职后2个会计年度内的记录
            in_period = (group['日期'] > leave_date) & (group['日期'] <= cutoff_date)
            df.loc[in_period.index, 'meu_9_3_condition'] = True
    
    # 3. 检查constraint条件 - 遵守减持限制
    # 假设董监高减持限制是: 每年减持不超过25%的持股量
    # 我们需要检查离任董监高是否遵守同样的限制
    
    # 首先计算每个股东在每个会计年度的累计减持比例
    df['会计年度减持累计'] = df.groupby(['公司简称', '股东', '会计年度'])['当日减持比例'].cumsum()
    
    # 标记违反减持限制的记录(减持超过25%)
    violates_limit = df['会计年度减持累计'] > 0.25
    df.loc[violates_limit, 'meu_9_3_constraint'] = False
    df.loc[~violates_limit, 'meu_9_3_constraint'] = True
    
    # 判断违规情况: 满足主体和条件但不满足约束
    is_violation = (
        df['meu_9_3_subject'] & 
        df['meu_9_3_condition'] & 
        (~df['meu_9_3_constraint'])
    )
    
    # 标记违规行
    df.loc[is_violation, 'meu_9_3_compliance'] = False
    
    return df

---

# Law Article 10

## MEU_10_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 |
| condition | 因涉嫌与本上市公司有关的证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1423 |
| completion_tokens | 933 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_10_1(df):
    '''
    检查上市公司大股东在被立案调查或侦查期间是否违规减持股份
    
    法律要求:
    "subject": "上市公司大股东", 
    "condition": "因涉嫌与本上市公司有关的证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间", 
    "constraint": "不得减持其所持有的本公司股份", 
    "contextual_info": NaN
    
    检查逻辑:
    1. 主体条件: 必须是上市公司大股东(包括控股股东、实际控制人或持股5%以上股东)
    2. 状态条件: 因涉嫌与本公司有关的证券期货违法犯罪被立案调查/侦查期间
    3. 约束条件: 在此期间不得减持股份
    '''
    
    # 初始化结果列
    df['meu_10_1_subject'] = False
    df['meu_10_1_condition'] = False
    df['meu_10_1_constraint'] = True  # 默认满足约束(未减持)
    df['meu_10_1_compliance'] = True   # 默认合规
    
    # 1. 检查主体条件: 上市公司大股东
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    df.loc[is_major_shareholder, 'meu_10_1_subject'] = True
    
    # 2. 检查状态条件: 被立案调查/侦查期间
    # 获取与本公司相关的调查/侦查事件
    investigation_events = [
        '被中国证监会及其派出机构立案调查',
        '被司法机关立案侦查'
    ]
    
    # 结束事件
    investigation_end_events = [
        '中国证监会及其派出机构立案调查结束',
        '司法机关立案侦查结束'
    ]
    
    # 按公司和股东分组，标记调查期间
    def mark_investigation_period(group):
        # 按日期排序
        group = group.sort_values('日期')
        
        # 初始化调查状态列
        group['under_investigation'] = False
        
        # 记录最近的调查开始和结束日期
        current_start = None
        
        for idx, row in group.iterrows():
            # 如果遇到新的调查开始事件
            if row['股东涉嫌证券期货违法犯罪事件'] in investigation_events:
                current_start = row['日期']
                group.loc[idx, 'under_investigation'] = True
            # 如果遇到调查结束事件且之前有调查
            elif (current_start is not None and 
                  row['股东涉嫌证券期货违法犯罪事件'] in investigation_end_events):
                # 标记从start到当前日期为调查期间
                mask = (group['日期'] >= current_start) & (group['日期'] <= row['日期'])
                group.loc[mask, 'under_investigation'] = True
                current_start = None
            # 如果当前处于调查期间
            elif current_start is not None:
                group.loc[idx, 'under_investigation'] = True
                
        return group
    
    # 按公司和股东分组处理
    df = df.groupby(['公司简称', '股东'], group_keys=False).apply(mark_investigation_period)
    
    # 标记满足条件的情况
    df.loc[df['under_investigation'], 'meu_10_1_condition'] = True
    
    # 3. 检查约束条件: 调查期间不得减持
    # 检查是否有减持行为 (当日减持比例 > 0)
    has_reduced = df['当日减持比例'] > 0
    
    # 标记违规减持情况 (满足主体和条件但减持了)
    violation = (
        is_major_shareholder & 
        df['under_investigation'] & 
        has_reduced
    )
    
    df.loc[violation, 'meu_10_1_constraint'] = False
    df.loc[violation, 'meu_10_1_compliance'] = False
    
    # 清理临时列
    df.drop('under_investigation', axis=1, inplace=True, errors='ignore')
    
    return df

---

## MEU_10_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 |
| condition | 因涉嫌与本上市公司有关的证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1434 |
| completion_tokens | 846 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_10_2(df):
    '''
    检查MEU_10_2合规性:
    "subject": "上市公司大股东", 
    "condition": "因涉嫌与本上市公司有关的证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月", 
    "constraint": "不得减持其所持有的本公司股份", 
    "contextual_info": NaN
    
    合规性逻辑:
    1. subject: 必须是上市公司大股东(控股股东/实际控制人/持股5%以上股东)
    2. condition: 因涉嫌证券期货违法犯罪被立案调查/侦查，且在行政处罚/刑事判决后未满6个月
    3. constraint: 不得减持股份
    '''

    # 初始化结果列
    df['meu_10_2_subject'] = False
    df['meu_10_2_condition'] = False
    df['meu_10_2_constraint'] = True  # 默认满足约束(未减持)
    df['meu_10_2_compliance'] = True  # 默认合规

    # 1. 验证subject: 上市公司大股东
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    df.loc[is_major_shareholder, 'meu_10_2_subject'] = True

    # 2. 验证condition: 被立案调查/侦查且在处罚后6个月内
    # 先按股东分组处理时间序列条件
    grouped = df.groupby(['公司简称', '股东'])
    
    # 记录每个股东最近一次处罚日期
    penalty_dates = pd.Series(index=df.index, dtype='datetime64[ns]')
    
    for (company, shareholder), group in grouped:
        # 获取该股东所有相关事件
        events = group['股东涉嫌证券期货违法犯罪事件'].dropna()
        
        # 找出最近的行政处罚/刑事判决日期
        last_penalty = None
        penalty_flags = events.isin(['行政处罚决定作出'])
        
        if penalty_flags.any():
            last_penalty = group.loc[penalty_flags.idxmax(), '日期']
        
        # 标记该股东在处罚后6个月内的所有日期
        if last_penalty is not None:
            mask = (group['日期'] >= last_penalty) & \
                   (group['日期'] <= last_penalty + pd.DateOffset(months=6))
            penalty_dates.loc[mask.index] = last_penalty
    
    # 合并回原DataFrame
    df['最近处罚日期'] = penalty_dates
    
    # 条件满足: 有处罚日期且在6个月内
    has_condition = ~df['最近处罚日期'].isna()
    df.loc[has_condition, 'meu_10_2_condition'] = True

    # 3. 验证constraint: 不得减持
    # 检查当日是否有减持行为(通过减持比例或减持数量判断)
    is_reducing = (
        (df['当日减持比例'] > 0) |  # 当日有减持比例
        (df['减持方式'].notna())    # 当日有减持方式记录
    )
    df.loc[is_reducing, 'meu_10_2_constraint'] = False

    # 判断违规情况: 满足主体和条件但不满足约束(即违规减持)
    is_violation = (
        is_major_shareholder & 
        has_condition & 
        is_reducing
    )
    df.loc[is_violation, 'meu_10_2_compliance'] = False

    return df

---

## MEU_10_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 |
| condition | 因涉及与本上市公司有关的违法违规，被本所公开谴责未满3个月 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1416 |
| completion_tokens | 909 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_10_3(df):
    '''
    检查MEU_10_3合规性:
    "subject": "上市公司大股东", 
    "condition": "因涉及与本上市公司有关的违法违规，被本所公开谴责未满3个月", 
    "constraint": "不得减持其所持有的本公司股份", 
    "contextual_info": NaN
    
    合规性检查逻辑:
    1. subject: 必须是上市公司大股东(控股股东/实际控制人/持股5%以上股东)
    2. condition: 该股东因涉及与本公司有关的违法违规被本所公开谴责未满3个月
    3. constraint: 在满足上述条件时不得减持股份
    '''
    
    # 初始化结果列
    df['meu_10_3_subject'] = False
    df['meu_10_3_condition'] = False
    df['meu_10_3_constraint'] = True  # 默认满足约束(未减持)
    df['meu_10_3_compliance'] = True  # 默认合规
    
    # 1. 检查subject条件: 上市公司大股东
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    df.loc[is_major_shareholder, 'meu_10_3_subject'] = True
    
    # 2. 检查condition条件: 被本所公开谴责未满3个月
    # 先找出所有被公开谴责的记录
    condemned = df[df['股东涉嫌证券期货违法犯罪事件'] == '被本所公开谴责']
    
    # 按股东分组，找出每个股东最近一次被谴责的日期
    if not condemned.empty:
        last_condemn_dates = condemned.groupby(['公司简称', '股东'])['日期'].max().reset_index()
        last_condemn_dates.columns = ['公司简称', '股东', '最后谴责日期']
        
        # 合并回原数据框
        df = pd.merge(df, last_condemn_dates, on=['公司简称', '股东'], how='left')
        
        # 计算谴责日期与当前日期的月份差
        df['谴责月份差'] = (df['日期'] - df['最后谴责日期']).dt.days / 30
        
        # 条件满足: 有谴责记录且未满3个月
        has_condemn = ~df['最后谴责日期'].isna()
        within_3months = df['谴责月份差'] < 3
        valid_condition = has_condemn & within_3months
        
        df.loc[valid_condition, 'meu_10_3_condition'] = True
        
        # 3. 检查constraint条件: 不得减持
        # 减持行为判断: 当日减持比例 > 0 或 累计减持比例增加
        # 需要按股东分组检查减持情况
        df['减持行为'] = False
        
        # 按股东分组处理
        for (company, shareholder), group in df.groupby(['公司简称', '股东']):
            # 计算每日减持比例变化
            group = group.sort_values('日期')
            group['减持比例变化'] = group['累计减持比例'].diff().fillna(0)
            
            # 标记有减持行为的日期
            has_reduced = (group['当日减持比例'] > 0) | (group['减持比例变化'] > 0)
            df.loc[has_reduced.index, '减持行为'] = has_reduced
            
        # 约束违反: 满足subject和condition但有减持行为
        violation = is_major_shareholder & valid_condition & df['减持行为']
        df.loc[violation, 'meu_10_3_constraint'] = False
        df.loc[violation, 'meu_10_3_compliance'] = False
        
        # 清理临时列
        df = df.drop(columns=['最后谴责日期', '谴责月份差', '减持行为'], errors='ignore')
    
    return df

---

## MEU_10_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 |
| condition | 因涉及证券期货违法，被中国证监会行政处罚且罚没款尚未足额缴纳，且不存在法律、行政法规另有规定或减持资金用于缴纳罚没款的情形 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1435 |
| completion_tokens | 604 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_10_4(df):
    '''
    检查上市公司大股东在被证监会行政处罚且罚没款未足额缴纳时不得减持股份的合规性
    
    参数:
        df: 包含公司和股东信息的DataFrame
        
    返回:
        添加了合规性检查结果的DataFrame
    
    法律要求:
        "subject": "上市公司大股东", 
        "condition": "因涉及证券期货违法，被中国证监会行政处罚且罚没款尚未足额缴纳，且不存在法律、行政法规另有规定或减持资金用于缴纳罚没款的情形", 
        "constraint": "不得减持其所持有的本公司股份", 
        "contextual_info": NaN
    '''
    
    # 初始化结果列
    df['meu_10_4_subject'] = False
    df['meu_10_4_condition'] = False
    df['meu_10_4_constraint'] = True  # 默认满足约束(即不减持)
    df['meu_10_4_compliance'] = True  # 默认合规
    
    # 1. 验证subject: 上市公司大股东
    # 大股东包括控股股东、实际控制人或持股5%以上股东
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人']) |
        (df['持股比例'] >= 0.05)
    )
    df.loc[is_major_shareholder, 'meu_10_4_subject'] = True
    
    # 2. 验证condition: 被行政处罚且罚没款未缴纳且无例外情形
    # 假设"行政处罚决定作出"表示已被行政处罚
    penalized = df['股东涉嫌证券期货违法犯罪事件'] == '行政处罚决定作出'
    
    # 假设没有"罚没款已足额缴纳"的标记字段，这里简化处理为只要被处罚就认为未缴纳
    # 且假设没有例外情形(没有相关字段)
    df.loc[is_major_shareholder & penalized, 'meu_10_4_condition'] = True
    
    # 3. 验证constraint: 不得减持股份
    # 检查是否有减持行为(当日减持比例>0)
    is_selling = df['当日减持比例'] > 0
    
    # 对于满足主体和条件的股东，如果减持则违反约束
    violation = (
        df['meu_10_4_subject'] & 
        df['meu_10_4_condition'] & 
        is_selling
    )
    df.loc[violation, 'meu_10_4_constraint'] = False
    df.loc[violation, 'meu_10_4_compliance'] = False
    
    return df

---

# Law Article 11

## MEU_11_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 |
| condition | 上市公司因涉嫌证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1426 |
| completion_tokens | 944 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_11_1(df):
    '''
    检查上市公司控股股东或实际控制人在公司涉嫌证券期货违法犯罪期间减持股份的合规性
    
    "subject": "上市公司控股股东 | 实际控制人", 
    "condition": "上市公司因涉嫌证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间", 
    "constraint": "不得减持其所持有的本公司股份", 
    "contextual_info": NaN
    
    实现逻辑：
    1. subject检查：股东身份是控股股东或实际控制人
    2. condition检查：公司处于被立案调查或侦查期间
    3. constraint检查：在condition满足期间没有减持行为
    '''
    
    # 初始化结果列
    df['meu_11_1_subject'] = False
    df['meu_11_1_condition'] = False
    df['meu_11_1_constraint'] = True  # 默认满足约束（没有减持）
    df['meu_11_1_compliance'] = True  # 默认合规
    
    # 1. 检查subject条件：控股股东或实际控制人
    valid_subject = df['股东身份'].isin(['控股股东', '实际控制人'])
    df.loc[valid_subject, 'meu_11_1_subject'] = True
    
    # 2. 检查condition条件：公司处于被立案调查或侦查期间
    # 定义违法犯罪事件开始和结束的标志
    investigation_start = df['公司涉嫌证券期货违法犯罪事件'].isin([
        '被中国证监会及其派出机构立案调查', 
        '被司法机关立案侦查'
    ])
    investigation_end = df['公司涉嫌证券期货违法犯罪事件'].isin([
        '中国证监会及其派出机构立案调查结束',
        '司法机关立案侦查结束',
        '行政处罚决定作出'
    ])
    
    # 按公司分组，标记调查期间
    df['investigation_start'] = investigation_start
    df['investigation_end'] = investigation_end
    
    # 为每个公司创建调查期间标记
    def mark_investigation_period(group):
        # 找出所有调查开始和结束的日期
        starts = group[group['investigation_start']]['日期']
        ends = group[group['investigation_end']]['日期']
        
        # 如果没有开始事件，直接返回全False
        if len(starts) == 0:
            group['in_investigation'] = False
            return group
        
        # 按时间排序开始和结束事件
        starts = starts.sort_values()
        ends = ends.sort_values()
        
        # 创建时间段列表
        periods = []
        current_start = None
        
        for date in group['日期']:
            # 检查是否在某个调查期间内
            in_period = False
            for i in range(len(starts)):
                # 如果有开始没有对应的结束，则一直持续
                if i >= len(ends) or starts.iloc[i] > ends.iloc[i]:
                    if date >= starts.iloc[i]:
                        in_period = True
                        break
                else:
                    if starts.iloc[i] <= date <= ends.iloc[i]:
                        in_period = True
                        break
            
            group.loc[group['日期'] == date, 'in_investigation'] = in_period
        
        return group
    
    # 按公司分组应用标记函数
    df = df.groupby('公司简称', group_keys=False).apply(mark_investigation_period)
    
    valid_condition = df['in_investigation']
    df.loc[valid_condition, 'meu_11_1_condition'] = True
    
    # 3. 检查constraint条件：在调查期间没有减持
    # 减持行为定义为当日减持比例 > 0
    has_reduction = df['当日减持比例'] > 0
    
    # 违规情况：满足subject和condition但有减持行为
    violation = valid_subject & valid_condition & has_reduction
    df.loc[violation, 'meu_11_1_constraint'] = False
    df.loc[violation, 'meu_11_1_compliance'] = False
    
    # 清理临时列
    df = df.drop(columns=['investigation_start', 'investigation_end', 'in_investigation'])
    
    return df

---

## MEU_11_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 |
| condition | 上市公司因涉嫌证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月的 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1437 |
| completion_tokens | 744 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_11_2(df):
    '''
    检查MEU_11_2合规性:
    "subject": "上市公司控股股东 | 实际控制人", 
    "condition": "上市公司因涉嫌证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月的", 
    "constraint": "不得减持其所持有的本公司股份", 
    "contextual_info": NaN
    
    实现逻辑:
    1. subject: 检查是否为控股股东或实际控制人
    2. condition: 检查公司是否在行政处罚/刑事判决后6个月内
    3. constraint: 检查是否有减持行为
    4. 违规情况: 满足subject和condition但不满足constraint(即在此期间减持)
    '''
    
    # 初始化结果列
    df['meu_11_2_subject'] = False
    df['meu_11_2_condition'] = False
    df['meu_11_2_constraint'] = True  # 默认满足约束(即没有减持)
    df['meu_11_2_compliance'] = True  # 默认合规
    
    # 1. 检查subject条件: 控股股东或实际控制人
    is_controlling_shareholder = df['股东身份'].isin(['控股股东', '实际控制人'])
    df['meu_11_2_subject'] = is_controlling_shareholder
    
    # 2. 检查condition条件: 公司在处罚/判决后6个月内
    # 先找出公司所有处罚/判决日期
    penalty_dates = df[df['公司涉嫌证券期货违法犯罪事件'].isin(['行政处罚决定作出'])]['日期'].unique()
    
    # 对每个公司分组处理
    grouped = df.groupby('公司简称')
    
    for company, group in grouped:
        # 获取该公司所有处罚日期
        company_penalty_dates = penalty_dates[penalty_dates.isin(group['日期'])]
        
        # 对每个处罚日期，计算6个月内的日期范围
        for penalty_date in company_penalty_dates:
            six_months_after = penalty_date + pd.DateOffset(months=6)
            condition_mask = (group['日期'] >= penalty_date) & (group['日期'] < six_months_after)
            
            # 标记condition满足的行
            df.loc[condition_mask.index, 'meu_11_2_condition'] = True
    
    # 3. 检查constraint条件: 是否有减持行为
    # 减持行为定义为: 当日减持比例 > 0 或 累计减持比例增加
    has_reduction = (df['当日减持比例'] > 0) | (df.groupby(['公司简称', '股东'])['累计减持比例'].diff() > 0)
    df.loc[has_reduction, 'meu_11_2_constraint'] = False
    
    # 4. 判断违规情况: 满足subject和condition但不满足constraint
    is_violation = (
        df['meu_11_2_subject'] & 
        df['meu_11_2_condition'] & 
        (~df['meu_11_2_constraint'])
    )
    
    df.loc[is_violation, 'meu_11_2_compliance'] = False
    
    return df

---

## MEU_11_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 |
| condition | 上市公司被本所公开谴责未满3个月 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1415 |
| completion_tokens | 682 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_11_3(df):
    '''
    检查MEU_11_3合规性:
    "subject": "上市公司控股股东 | 实际控制人", 
    "condition": "上市公司被本所公开谴责未满3个月", 
    "constraint": "不得减持其所持有的本公司股份"
    
    参数:
    df - 包含公司和股东信息的DataFrame
    
    返回:
    标注合规性后的DataFrame
    '''
    
    # 初始化结果列
    df['meu_11_3_subject'] = False
    df['meu_11_3_condition'] = False
    df['meu_11_3_constraint'] = False
    df['meu_11_3_compliance'] = True  # 默认合规
    
    # 1. 验证subject: 控股股东或实际控制人
    is_controlling_shareholder = df['股东身份'].isin(['控股股东', '实际控制人'])
    df['meu_11_3_subject'] = is_controlling_shareholder
    
    # 2. 验证condition: 公司被公开谴责未满3个月
    # 先找出公司被公开谴责的日期
    condemned_companies = df[df['公司涉嫌证券期货违法犯罪事件'] == '被本所公开谴责']
    
    if not condemned_companies.empty:
        # 对每个公司计算谴责后3个月的日期
        for company in condemned_companies['公司简称'].unique():
            # 获取该公司最近一次被谴责的日期
            last_condemn_date = condemned_companies[condemned_companies['公司简称'] == company]['日期'].max()
            # 计算3个月后的日期
            three_months_later = last_condemn_date + pd.DateOffset(months=3)
            
            # 标记该公司在谴责后3个月内的日期
            company_mask = (
                (df['公司简称'] == company) & 
                (df['日期'] >= last_condemn_date) & 
                (df['日期'] <= three_months_later)
            )
            df.loc[company_mask, 'meu_11_3_condition'] = True
    
    # 3. 验证constraint: 不得减持股份
    # 减持行为定义为: 当日减持比例 > 0 或 存在减持计划
    is_reducing = (df['当日减持比例'] > 0) | df['存在减持计划']
    df['meu_11_3_constraint'] = ~is_reducing  # 约束是不得减持，所以不减持为True
    
    # 4. 综合判断合规性
    # 违规情况: 满足主体和条件但违反约束(即减持)
    violation_mask = (
        df['meu_11_3_subject'] & 
        df['meu_11_3_condition'] & 
        (~df['meu_11_3_constraint'])  # 即减持了
    )
    df.loc[violation_mask, 'meu_11_3_compliance'] = False
    
    return df

---

## MEU_11_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 |
| condition | 市公司股票因可能触及重大违法强制退市情形，而被本所实施退市风险警示，在本所规定的限制减持期限内的 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1434 |
| completion_tokens | 507 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_11_4(df):
    '''
    检查上市公司控股股东或实际控制人在公司股票因可能触及重大违法强制退市情形而被实施退市风险警示期间，
    是否违反不得减持股份的规定。
    
    参数:
        df: 包含公司和股东信息的DataFrame
        
    返回:
        标注合规性后的DataFrame
    '''
    
    # 初始化合规性检查列
    df['meu_11_4_subject'] = False  # 主体是否符合
    df['meu_11_4_condition'] = False  # 条件是否符合
    df['meu_11_4_constraint'] = True  # 约束是否满足(默认满足)
    df['meu_11_4_compliance'] = True  # 总体合规性(默认合规)
    
    # 1. 检查主体: 上市公司控股股东或实际控制人
    is_controlling_shareholder = df['股东身份'].isin(['控股股东', '实际控制人'])
    df.loc[is_controlling_shareholder, 'meu_11_4_subject'] = True
    
    # 2. 检查条件: 公司股票因可能触及重大违法强制退市情形而被实施退市风险警示
    # 假设有列'退市风险警示状态'表示是否被实施退市风险警示
    # 如果没有该列，可能需要从其他信息推断，这里假设存在该列
    under_warning = df['公司涉嫌证券期货违法犯罪事件'] == '被中国证监会及其派出机构立案调查'
    df.loc[under_warning, 'meu_11_4_condition'] = True
    
    # 3. 检查约束: 不得减持股份
    # 检查是否有减持行为(当日减持比例>0)
    has_reduction = df['当日减持比例'] > 0
    df.loc[has_reduction, 'meu_11_4_constraint'] = False
    
    # 总体合规性: 满足主体和条件但不满足约束即为违规
    is_violation = (
        is_controlling_shareholder & 
        under_warning & 
        has_reduction
    )
    df.loc[is_violation, 'meu_11_4_compliance'] = False
    
    return df

---

# Law Article 12

## MEU_12_1


| 字段 | 内容 |
|------|------|
| subject | 公开发行股票并上市时的控股股东 | 公开发行股票并上市时的实际控制人 |
| condition | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constraint | 不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格 |
| contextual_info | 股票收盘价以公开发行股票并上市之日为基准向后复权计算 |
| note | 不考虑中国证监会另有规定的情况 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1490 |
| completion_tokens | 1018 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_12_1(df):
    """
    检查MEU_12_1合规性:
    "subject": "公开发行股票并上市时的控股股东 | 公开发行股票并上市时的实际控制人",
    "condition": "计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的",
    "constraint": "不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格",
    "contextual_info": "股票收盘价以公开发行股票并上市之日为基准向后复权计算"
    
    处理步骤:
    1. 确定发行价格和发行时的控股股东/实际控制人
    2. 验证subject条件
    3. 验证condition条件
    4. 验证constraint条件
    5. 标记合规情况
    """
    
    # 初始化结果列
    df['meu_12_1_subject'] = False
    df['meu_12_1_condition'] = False
    df['meu_12_1_constraint'] = True  # 默认满足约束(没有股价低于发行价)
    df['meu_12_1_compliance'] = True  # 默认合规
    
    # 按公司和股东分组处理
    grouped = df.groupby(['公司简称', '股东'])
    
    for (company, shareholder), group in grouped:
        # 步骤1: 确定发行价格和发行时的股东身份
        # 找到第一个非空的收盘价日期作为上市日期
        first_trade_date = group['日期'].min()
        issue_price = group.loc[group['日期'] == first_trade_date, '收盘价'].values[0]
        
        # 获取发行时的股东身份
        initial_status = group.loc[group['日期'] == first_trade_date, '股东身份'].values[0]
        
        # 步骤2: 验证subject条件 - 必须是发行时的控股股东或实际控制人
        is_subject = initial_status in ['控股股东', '实际控制人']
        
        # 步骤3: 验证condition条件
        # 条件1: 减持方式是集中竞价或大宗交易
        valid_trade_method = group['减持方式'].isin(['竞价交易', '大宗交易'])
        
        # 条件2: 是首次披露减持计划(即之前没有减持计划)
        # 计算每个减持计划是否是首次
        has_plan = group['存在减持计划']
        first_plan_date = group.loc[has_plan, '日期'].min() if has_plan.any() else None
        
        # 标记首次披露的减持计划
        is_first_plan = (group['日期'] == first_plan_date) if first_plan_date else False
        
        valid_condition = valid_trade_method & is_first_plan
        
        # 步骤4: 验证constraint条件
        # 对于每个交易日，检查前20个交易日内是否有收盘价低于发行价
        for idx, row in group.iterrows():
            # 获取当前日期前20个交易日的收盘价
            prev_20_days = group[
                (group['日期'] <= row['日期']) & 
                (group['日期'] > row['日期'] - pd.Timedelta(days=20))
            ]
            
            # 检查是否有任何收盘价低于发行价(考虑复权)
            if not prev_20_days.empty:
                # 计算复权后的收盘价(假设复权因子已经正确计算)
                adjusted_prices = prev_20_days['收盘价'] * prev_20_days['复权因子']
                price_below_issue = (adjusted_prices < issue_price).any()
                
                # 标记constraint满足情况
                if price_below_issue:
                    df.loc[idx, 'meu_12_1_constraint'] = False
        
        # 标记subject和condition满足情况
        df.loc[group.index, 'meu_12_1_subject'] = is_subject
        df.loc[group.index, 'meu_12_1_condition'] = valid_condition
        
    # 综合判断合规情况
    # 违规情况: 满足subject和condition但不满足constraint
    is_violation = (
        df['meu_12_1_subject'] & 
        df['meu_12_1_condition'] & 
        (~df['meu_12_1_constraint'])
    )
    
    df.loc[is_violation, 'meu_12_1_compliance'] = False
    
    return df

---

## MEU_12_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 | 一致行动人 |
| condition | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constraint | 不得存在下列情形：最近20个交易日内，上市公司任一交易日股票收盘价低于最近一个会计年度或者最近一期财务会计报告期末每股归属于上市公司股东的净资产 |
| contextual_info | 股票收盘价以最近一个会计年度或者最近一期财务会计报告资产负债表日为基准分别向后复权计算 |
| note | 不考虑中国证监会另有规定的情况 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1499 |
| completion_tokens | 948 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_12_2(df):
    '''
    检查MEU_12_2合规性:
    "subject": "上市公司控股股东 | 实际控制人 | 一致行动人", 
    "condition": "计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的", 
    "constraint": "不得存在下列情形：最近20个交易日内，上市公司任一交易日股票收盘价低于最近一个会计年度或者最近一期财务会计报告期末每股归属于上市公司股东的净资产", 
    "contextual_info": "股票收盘价以最近一个会计年度或者最近一期财务会计报告资产负债表日为基准分别向后复权计算"
    
    不考虑中国证监会另有规定的情况
    '''
    
    # 初始化结果列
    df['meu_12_2_subject'] = False
    df['meu_12_2_condition'] = False
    df['meu_12_2_constraint'] = True  # 默认满足约束(没有股价低于净资产)
    df['meu_12_2_compliance'] = True  # 默认合规
    
    # 1. 验证subject: 控股股东/实际控制人/一致行动人
    valid_subject = df['股东身份'].isin(['控股股东', '实际控制人', '一致行动人'])
    df.loc[valid_subject, 'meu_12_2_subject'] = True
    
    # 2. 验证condition: 
    # (1)减持方式是集中竞价或大宗交易
    valid_trade_method = df['减持方式'].isin(['竞价交易', '大宗交易'])
    # (2)是首次披露减持计划(不存在已经按照本指引第四条披露减持计划)
    # 假设"存在减持计划"为True表示首次披露
    valid_first_disclosure = df['存在减持计划'] 
    # 合并条件
    valid_condition = valid_trade_method & valid_first_disclosure
    df.loc[valid_condition, 'meu_12_2_condition'] = True
    
    # 3. 验证constraint: 最近20个交易日没有股价低于每股净资产
    # 需要按公司和日期分组计算
    
    # 先复制一份数据用于计算
    constraint_df = df[['日期', '公司简称', '收盘价', '每股净资产', '复权因子']].copy()
    
    # 计算复权后的收盘价和每股净资产
    constraint_df['复权收盘价'] = constraint_df['收盘价'] * constraint_df['复权因子']
    constraint_df['复权每股净资产'] = constraint_df['每股净资产'] * constraint_df['复权因子']
    
    # 对每个公司计算最近20个交易日是否有股价低于净资产
    def check_price_below_na(group):
        # 按日期排序
        group = group.sort_values('日期')
        # 计算滚动20个交易日内的最低股价是否低于净资产
        group['20日最低价'] = group['复权收盘价'].rolling(20, min_periods=1).min()
        group['最近每股净资产'] = group['复权每股净资产'].shift(1).ffill()
        group['股价低于净资产'] = group['20日最低价'] < group['最近每股净资产']
        return group
    
    # 按公司分组应用检查
    constraint_df = constraint_df.groupby('公司简称').apply(check_price_below_na)
    
    # 合并回原DataFrame
    df = df.merge(constraint_df[['日期', '公司简称', '股价低于净资产']], 
                 on=['日期', '公司简称'], how='left')
    
    # 标记constraint违规(存在股价低于净资产)
    df.loc[df['股价低于净资产'], 'meu_12_2_constraint'] = False
    
    # 综合判断合规性: 满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        df['股价低于净资产']
    )
    
    # 标记违规行
    df.loc[is_violation, 'meu_12_2_compliance'] = False
    
    # 清理临时列
    df.drop(columns=['股价低于净资产'], inplace=True, errors='ignore')
    
    return df

---

## MEU_12_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 | 一致行动人 |
| condition | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constraint | 不得存在下列情形：上市公司最近一期经审计的财务报告的归属于上市公司股东的净利润为负 |
| contextual_info | nan |
| note | 不考虑中国证监会另有规定的情况 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1464 |
| completion_tokens | 851 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_12_3(df):
    '''
    检查MEU_12_3合规性:
    "subject": "上市公司控股股东 | 实际控制人 | 一致行动人", 
    "condition": "计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的", 
    "constraint": "不得存在下列情形：上市公司最近一期经审计的财务报告的归属于上市公司股东的净利润为负", 
    "contextual_info": NaN
    
    处理逻辑:
    1. 验证主体: 控股股东/实际控制人/一致行动人
    2. 验证条件: 有减持计划且是首次披露(无历史披露记录)
    3. 验证约束: 公司最近一期净利润不为负
    '''
    
    # 初始化结果列
    df['meu_12_3_subject'] = False
    df['meu_12_3_condition'] = False
    df['meu_12_3_constraint'] = False
    df['meu_12_3_compliance'] = True  # 默认合规
    
    # 1. 验证subject: 控股股东/实际控制人/一致行动人
    valid_subject = df['股东身份'].isin(['控股股东', '实际控制人', '一致行动人'])
    df.loc[valid_subject, 'meu_12_3_subject'] = True
    
    # 2. 验证condition: 
    # (1)减持方式为集中竞价或大宗交易
    valid_trade_type = df['减持方式'].isin(['竞价交易', '大宗交易'])
    
    # (2)首次披露减持计划(需要按股东分组检查是否为首次披露)
    # 先按股东分组，标记每个股东的首次披露日
    first_disclosure = df[df['存在减持计划']].groupby(['公司简称', '股东'])['计划披露日'].min().reset_index()
    first_disclosure['is_first'] = True
    
    # 合并回原df
    df = df.merge(first_disclosure, 
                 on=['公司简称', '股东', '计划披露日'],
                 how='left',
                 suffixes=('', '_first'))
    
    # 首次披露且当前有减持计划
    valid_first_disclosure = (df['is_first'] == True) & df['存在减持计划']
    
    # 综合条件
    valid_condition = valid_trade_type & valid_first_disclosure
    df.loc[valid_condition, 'meu_12_3_condition'] = True
    
    # 3. 验证constraint: 最近一期净利润不为负
    # 获取每个公司最近一期的净利润(按日期降序取第一个)
    latest_financials = df.sort_values(['公司简称', '日期'], ascending=[True, False])\
                         .groupby('公司简称').first().reset_index()
    
    # 创建净利润是否为正的映射
    profit_status = latest_financials[['公司简称', '净利润']].set_index('公司简称')['净利润'] >= 0
    
    # 合并回原df
    df['latest_profit_positive'] = df['公司简称'].map(profit_status)
    valid_constraint = df['latest_profit_positive']
    df.loc[valid_constraint, 'meu_12_3_constraint'] = True
    
    # 判断违规情况: 满足主体和条件但不满足约束
    is_violation = valid_subject & valid_condition & (~valid_constraint)
    df.loc[is_violation, 'meu_12_3_compliance'] = False
    
    # 清理临时列
    df.drop(['is_first', 'latest_profit_positive'], axis=1, inplace=True, errors='ignore')
    
    return df

---

## MEU_12_4


| 字段 | 内容 |
|------|------|
| subject | 控股股东 | 实际控制人（上市后不再具备相关主体身份的） |
| condition | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constraint | 不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格 |
| contextual_info | 股票收盘价以公开发行股票并上市之日为基准向后复权计算 |
| note | 不考虑中国证监会另有规定的情况 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1487 |
| completion_tokens | 1077 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_12_4(df):
    '''
    检查MEU_12_4合规性:
    "subject": "控股股东 | 实际控制人（上市后不再具备相关主体身份的）", 
    "condition": "计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的", 
    "constraint": "不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格", 
    "contextual_info": "股票收盘价以公开发行股票并上市之日为基准向后复权计算"
    
    处理逻辑:
    1. 识别符合条件的subject(控股股东或实际控制人，包括上市后不再具备相关身份的)
    2. 检查condition(首次披露减持计划且通过集中竞价或大宗交易减持)
    3. 验证constraint(最近20个交易日收盘价不低于发行价)
    '''
    
    # 初始化结果列
    df['meu_12_4_subject'] = False
    df['meu_12_4_condition'] = False
    df['meu_12_4_constraint'] = True  # 默认满足约束
    df['meu_12_4_compliance'] = True  # 默认合规
    
    # 1. 验证subject条件 - 控股股东或实际控制人(包括上市后不再具备相关身份的)
    is_controlling = df['股东身份'].isin(['控股股东', '实际控制人'])
    
    # 获取每个公司最早的上市日期和发行价格
    company_info = df.groupby('公司简称').agg({
        '上市日期': 'first',
        '发行价格': 'first'
    }).reset_index()
    
    # 合并发行信息到主表
    df = pd.merge(df, company_info, on='公司简称', suffixes=('', '_company'))
    
    # 标记上市后不再具备相关身份的股东
    # 假设股东如果在上市日期后不再是控股股东或实际控制人，但仍保留股东身份
    # 这里简化处理：只要曾经是控股股东或实际控制人，且当前有持股
    df['曾经是控股股东或实际控制人'] = (
        (df['股东身份'].isin(['控股股东', '实际控制人'])) | 
        (df['股东'].isin(df[df['股东身份'].isin(['控股股东', '实际控制人'])]['股东'].unique()))
    )
    
    valid_subject = is_controlling | df['曾经是控股股东或实际控制人']
    
    # 2. 验证condition条件
    # 首次披露减持计划且通过集中竞价或大宗交易减持
    # 计算是否是首次披露减持计划
    df['减持计划序号'] = df.groupby(['公司简称', '股东'])['存在减持计划'].cumsum()
    is_first_plan = df['减持计划序号'] == 1
    
    valid_condition = (
        is_first_plan & 
        df['存在减持计划'] & 
        df['减持方式'].isin(['竞价交易', '大宗交易'])
    )
    
    # 3. 验证constraint条件
    # 最近20个交易日内任一交易日收盘价不低于发行价格
    
    # 计算复权后的收盘价 (假设复权因子已经正确计算)
    df['复权收盘价'] = df['收盘价'] * df['复权因子']
    
    # 对每个公司和日期，检查前20个交易日的收盘价
    for idx, row in df[valid_subject & valid_condition].iterrows():
        company = row['公司简称']
        date = row['日期']
        issue_price = row['发行价格_company']
        
        # 获取该日期前20个交易日的收盘价
        prev_prices = df[
            (df['公司简称'] == company) & 
            (df['日期'] <= date) & 
            (df['日期'] > date - pd.Timedelta(days=20))
        ]['复权收盘价']
        
        # 检查是否有任何一天的收盘价低于发行价
        if not prev_prices.empty and (prev_prices < issue_price).any():
            df.loc[idx, 'meu_12_4_constraint'] = False
    
    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_12_4_subject'] = True
    df.loc[valid_condition, 'meu_12_4_condition'] = True
    
    # 判断违规情况：满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~df['meu_12_4_constraint'])
    )
    
    # 标记违规行
    df.loc[is_violation, 'meu_12_4_compliance'] = False
    
    return df

---

## MEU_12_5


| 字段 | 内容 |
|------|------|
| subject | 公开发行时持股5%以上的第一大股东 | 一致行动人 |
| condition | 上市公司在公开发行股票并上市时披露为无控股股东、实际控制人, 且计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constraint | 不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格 |
| contextual_info | 股票收盘价以公开发行股票并上市之日为基准向后复权计算 |
| note | 不考虑中国证监会另有规定的情况 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1505 |
| completion_tokens | 1075 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_12_5(df):
    '''
    检查MEU_12_5合规性:
    "subject": "公开发行时持股5%以上的第一大股东 | 一致行动人",
    "condition": "上市公司在公开发行股票并上市时披露为无控股股东、实际控制人, 且计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的",
    "constraint": "不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格",
    "contextual_info": "股票收盘价以公开发行股票并上市之日为基准向后复权计算"
    '''
    
    # 初始化结果列
    df['meu_12_5_subject'] = False
    df['meu_12_5_condition'] = False
    df['meu_12_5_constraint'] = True  # 默认满足约束(没有股价低于发行价)
    df['meu_12_5_compliance'] = True  # 默认合规
    
    # 1. 验证subject条件: 公开发行时持股5%以上的第一大股东或一致行动人
    # 获取每个公司公开发行时的第一大股东(持股比例最高)
    first_day = df.groupby('公司简称')['日期'].transform('min') == df['日期']
    major_shareholders = df[first_day & (df['持股比例'] >= 0.05)]
    top_shareholders = major_shareholders.loc[major_shareholders.groupby('公司简称')['持股比例'].idxmax()]
    
    # 标记所有属于公开发行时第一大股东或一致行动人的行
    is_top_shareholder = df['股东'].isin(top_shareholders['股东'])
    is_major_shareholder = df['持股比例'] >= 0.05
    valid_subject = is_top_shareholder | is_major_shareholder
    
    # 2. 验证condition条件:
    # a) 公司披露为无控股股东、实际控制人
    no_controller = ~df['股东身份'].isin(['控股股东', '实际控制人'])
    
    # b) 计划通过集中竞价或大宗交易减持且首次披露减持计划
    valid_trade_type = df['减持方式'].isin(['竞价交易', '大宗交易'])
    first_disclosure = df.groupby(['公司简称', '股东'])['存在减持计划'].cumsum() == 1
    
    # c) 不存在已经按照第四条披露减持计划的情况(假设第四条是MEU_4_1)
    # 这里简化处理: 只要不是首次披露就认为不符合
    valid_condition = no_controller & valid_trade_type & first_disclosure & df['存在减持计划']
    
    # 3. 验证constraint条件: 最近20个交易日内股价不低于发行价
    # 计算复权后的收盘价(假设复权因子已经正确计算)
    df['复权收盘价'] = df['收盘价'] * df['复权因子']
    
    # 获取发行价(每个公司最早的非空收盘价)
    df['发行价'] = df.groupby('公司简称')['收盘价'].transform('first')
    
    # 对每个公司-股东组合,检查最近20个交易日是否有股价低于发行价
    def check_price_constraint(group):
        # 获取当前日期前20个交易日
        current_date = group.name[2]  # 假设多级索引是(公司,股东,日期)
        last_20 = (df['日期'] <= current_date) & (df['日期'] > current_date - pd.Timedelta(days=20))
        company_stock = df[last_20 & (df['公司简称'] == group.name[0])]
        has_low_price = (company_stock['复权收盘价'] < company_stock['发行价'].iloc[0]).any()
        return has_low_price
    
    # 分组应用约束检查(优化性能,避免逐行检查)
    grouped = df.groupby(['公司简称', '股东', '日期'])
    price_violation = grouped.apply(check_price_constraint).reset_index(level=[0,1,2], drop=True)
    df['meu_12_5_constraint'] = ~price_violation
    
    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_12_5_subject'] = True
    df.loc[valid_condition, 'meu_12_5_condition'] = True
    
    # 判断违规情况: 满足主体和条件但不满足约束
    is_violation = valid_subject & valid_condition & price_violation
    df.loc[is_violation, 'meu_12_5_compliance'] = False
    
    return df

---

# Law Article 13

## MEU_13_2


| 字段 | 内容 |
|------|------|
| subject | 大股东 |
| condition | 通过协议转让方式减持股份导致出让方不再具有大股东身份, 且处在减持后6个月内 |
| constraint | 继续遵守本指引第四条规定 |
| contextual_info | nan |
| note | nan |
| relation | should_include |
| target | Law_4 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1421 |
| completion_tokens | 906 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_13_2(df):
    '''
    检查MEU_13_2合规性:
    "subject": "大股东", 
    "condition": "通过协议转让方式减持股份导致出让方不再具有大股东身份, 且处在减持后6个月内", 
    "constraint": "继续遵守本指引第四条规定", 
    "contextual_info": NaN
    
    处理逻辑:
    1. 先识别大股东(持股5%以上)
    2. 检查是否通过协议转让减持导致不再是大股东
    3. 检查减持后6个月内是否遵守第四条(假设第四条是关于减持比例限制等)
    '''
    
    # 初始化结果列
    df['meu_13_2_subject'] = False
    df['meu_13_2_condition'] = False
    df['meu_13_2_constraint'] = False
    df['meu_13_2_compliance'] = True
    
    # 1. 识别大股东 (持股5%以上或身份为持股5%以上股东)
    is_major_shareholder = (
        (df['持股比例'] >= 0.05) | 
        df['股东身份'].isin(['持股5%以上股东', '控股股东', '实际控制人'])
    )
    df.loc[is_major_shareholder, 'meu_13_2_subject'] = True
    
    # 2. 检查协议转让导致不再是大股东的条件
    # 需要分组处理每个股东的状态变化
    grouped = df.groupby(['公司简称', '股东'])
    
    for (company, shareholder), group in grouped:
        # 按日期排序
        group = group.sort_values('日期')
        
        # 找出协议转让导致持股比例低于5%的日期
        for i in range(1, len(group)):
            prev_row = group.iloc[i-1]
            curr_row = group.iloc[i]
            
            # 检查是否通过协议转让减持且导致不再是5%以上股东
            condition_met = (
                prev_row['meu_13_2_subject'] and  # 之前是大股东
                curr_row['减持方式'] == '协议转让' and  # 通过协议转让减持
                curr_row['持股比例'] < 0.05 and  # 减持后持股低于5%
                (curr_row['日期'] - prev_row['日期']).days <= 1  # 确保是连续交易日
            )
            
            if condition_met:
                # 标记减持日及之后6个月内(约180天)的所有记录
                end_date = curr_row['日期'] + pd.Timedelta(days=180)
                mask = (
                    (df['公司简称'] == company) & 
                    (df['股东'] == shareholder) & 
                    (df['日期'] >= curr_row['日期']) & 
                    (df['日期'] <= end_date)
                )
                df.loc[mask, 'meu_13_2_condition'] = True
                
                # 3. 检查减持后6个月内是否遵守第四条(这里假设第四条是关于减持比例限制)
                # 示例检查: 减持后6个月内是否有违规减持行为
                # 实际应根据第四条具体内容实现
                violation_mask = mask & (
                    (df['当日减持比例'] > 0.01) |  # 假设第四条限制单日减持不超过1%
                    (df['累计减持比例'] > 0.05)   # 假设第六条限制累计减持不超过5%
                )
                df.loc[violation_mask, 'meu_13_2_constraint'] = False
                df.loc[~violation_mask & mask, 'meu_13_2_constraint'] = True
                
                # 标记违规情况(满足主体和条件但不满足约束)
                violation = mask & (
                    df['meu_13_2_subject'] & 
                    df['meu_13_2_condition'] & 
                    (~df['meu_13_2_constraint'])
                )
                df.loc[violation, 'meu_13_2_compliance'] = False
    
    return df

---

## MEU_13_3


| 字段 | 内容 |
|------|------|
| subject | 控股股东 | 实际控制人 |
| condition | 通过协议转让方式减持股份导致其不再具有控股股东、实际控制人身份, 且处于减持后的6个月内 |
| constraint | 应当继续遵守本指引第十二条第一款第二、三项规定 |
| contextual_info | nan |
| note | nan |
| relation | should_include |
| target | MEU_12_1;MEU_12_2 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1435 |
| completion_tokens | 780 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_13_3(df):
    '''
    检查MEU_13_3合规性:
    "subject": "控股股东 | 实际控制人", 
    "condition": "通过协议转让方式减持股份导致其不再具有控股股东、实际控制人身份, 且处于减持后的6个月内", 
    "constraint": "应当继续遵守本指引第十二条第一款第二、三项规定", 
    "contextual_info": NaN
    
    实现逻辑:
    1. subject: 检查是否为控股股东或实际控制人
    2. condition: 
       a) 通过协议转让方式减持
       b) 减持导致不再具有控股股东/实际控制人身份
       c) 处于减持后6个月内
    3. constraint: 需要引用其他条款(暂不实现)
    '''
    
    # 初始化结果列
    df['meu_13_3_subject'] = False
    df['meu_13_3_condition'] = False
    df['meu_13_3_compliance'] = True  # 默认合规
    
    # 1. 检查subject: 控股股东或实际控制人
    is_controlling = df['股东身份'].isin(['控股股东', '实际控制人'])
    df['meu_13_3_subject'] = is_controlling
    
    # 2. 检查condition
    # 按公司和股东分组处理
    grouped = df.groupby(['公司简称', '股东'])
    
    for (company, shareholder), group in grouped:
        # 只处理控股股东或实际控制人
        if not group['meu_13_3_subject'].any():
            continue
            
        # 找出身份变更的日期(不再具有控股股东/实际控制人身份)
        status_change_dates = group[group['股东身份'].isin(['控股股东', '实际控制人'])].groupby('日期').first()
        if len(status_change_dates) == 0:
            continue
            
        # 找出通过协议转让减持的日期
        protocol_transfer = group[group['减持方式'] == '协议转让']
        
        # 检查每次减持是否导致身份变更
        for date, row in protocol_transfer.iterrows():
            # 检查减持后是否仍为控股股东/实际控制人
            post_transfer = group[group['日期'] > date]
            if len(post_transfer) == 0:
                continue
                
            next_status = post_transfer.iloc[0]['股东身份']
            if next_status in ['控股股东', '实际控制人']:
                continue  # 身份未变更
                
            # 计算减持后6个月的日期范围
            six_months_later = date + pd.DateOffset(months=6)
            condition_period = (group['日期'] >= date) & (group['日期'] <= six_months_later)
            
            # 标记满足条件的行
            df.loc[condition_period & (df['公司简称'] == company) & 
                  (df['股东'] == shareholder), 'meu_13_3_condition'] = True
    
    # 3. 检查违规情况(满足subject和condition但未遵守约束)
    # 注意: 由于约束条款引用其他条款，此处仅标记需要进一步检查的行
    needs_check = df['meu_13_3_subject'] & df['meu_13_3_condition']
    df.loc[needs_check, 'meu_13_3_compliance'] = '需要检查第十二条合规性'
    
    return df

---

# Law Article 14

## MEU_14_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司因涉嫌证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1422 |
| completion_tokens | 553 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_1(df):
    '''
    检查上市公司董监高在被立案调查或侦查期间是否违规减持股份
    
    "subject": "上市公司董监高", 
    "condition": "上市公司因涉嫌证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间", 
    "constraint": "不得减持其所持有的本公司股份", 
    "contextual_info": NaN
    
    合规性检查逻辑:
    1. subject: 必须是上市公司董监高(不包括离任董监高)
    2. condition: 公司正处于被立案调查或侦查期间
    3. constraint: 在此期间不得减持股份
    '''

    # 初始化结果列
    df['meu_14_1_subject'] = False
    df['meu_14_1_condition'] = False
    df['meu_14_1_constraint'] = True  # 默认满足约束(未减持)
    df['meu_14_1_compliance'] = True  # 默认合规

    # 1. 检查subject条件: 必须是上市公司董监高(不包括离任董监高)
    is_director = df['股东身份'] == '董监高'
    df.loc[is_director, 'meu_14_1_subject'] = True

    # 2. 检查condition条件: 公司正处于被立案调查或侦查期间
    investigation_status = [
        '被中国证监会及其派出机构立案调查',
        '被司法机关立案侦查'
    ]
    is_under_investigation = df['公司涉嫌证券期货违法犯罪事件'].isin(investigation_status)
    df.loc[is_under_investigation, 'meu_14_1_condition'] = True

    # 3. 检查constraint条件: 在此期间不得减持股份
    # 计算当日是否有减持行为(通过减持比例或持股数量变化判断)
    # 这里假设如果当日减持比例>0则表示有减持行为
    has_reduced = df['当日减持比例'] > 0
    
    # 标记违规情况: 满足主体和条件但有减持行为
    is_violation = (
        is_director & 
        is_under_investigation & 
        has_reduced
    )
    
    df.loc[is_violation, 'meu_14_1_constraint'] = False
    df.loc[is_violation, 'meu_14_1_compliance'] = False

    return df

---

## MEU_14_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司因涉嫌证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1433 |
| completion_tokens | 633 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_2(df):
    '''
    检查MEU_14_2合规性:
    "subject": "上市公司董监高", 
    "condition": "上市公司因涉嫌证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月", 
    "constraint": "不得减持其所持有的本公司股份", 
    "contextual_info": NaN
    
    实现逻辑:
    1. subject: 检查是否为现任董监高
    2. condition: 检查公司是否在立案调查/侦查后且行政处罚/刑事判决未满6个月
    3. constraint: 检查是否有减持行为
    '''
    
    # 初始化结果列
    df['meu_14_2_subject'] = False
    df['meu_14_2_condition'] = False
    df['meu_14_2_constraint'] = True  # 默认满足约束(未减持)
    df['meu_14_2_compliance'] = True  # 默认合规
    
    # 1. 检查subject条件: 必须是现任董监高
    is_director = df['股东身份'] == '董监高'
    df.loc[is_director, 'meu_14_2_subject'] = True
    
    # 2. 检查condition条件: 公司处于处罚后6个月内
    # 先找出每个公司最近一次行政处罚/刑事判决的日期
    penalty_dates = df[df['公司涉嫌证券期货违法犯罪事件'].isin(['行政处罚决定作出'])].groupby('公司简称')['日期'].max()
    
    # 为每行数据标记是否在处罚后6个月内
    def is_within_6months(row):
        company = row['公司简称']
        if company in penalty_dates:
            penalty_date = penalty_dates[company]
            return (row['日期'] - penalty_date).days < 180  # 6个月=180天
        return False
    
    within_penalty_period = df.apply(is_within_6months, axis=1)
    df.loc[within_penalty_period, 'meu_14_2_condition'] = True
    
    # 3. 检查constraint条件: 是否有减持行为
    # 检查当日是否有减持(当日减持比例>0)
    has_sold = df['当日减持比例'] > 0
    df.loc[has_sold, 'meu_14_2_constraint'] = False
    
    # 综合判断违规情况: 满足主体和条件但不满足约束
    is_violation = (
        is_director & 
        within_penalty_period & 
        has_sold
    )
    
    df.loc[is_violation, 'meu_14_2_compliance'] = False
    
    return df

---

## MEU_14_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 本人因涉嫌与该上市公司有关的证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1425 |
| completion_tokens | 491 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_3(df):
    '''
    检查MEU_14_3合规性:
    "subject": "上市公司董监高", 
    "condition": "本人因涉嫌与该上市公司有关的证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间", 
    "constraint": "不得减持其所持有的本公司股份"
    
    合规性逻辑:
    1. 主体必须是上市公司董监高(不包括离任董监高)
    2. 条件是被立案调查/侦查期间
    3. 约束是不得减持股份
    '''
    
    # 初始化结果列
    df['meu_14_3_subject'] = False
    df['meu_14_3_condition'] = False
    df['meu_14_3_constraint'] = True  # 默认满足约束(未减持)
    df['meu_14_3_compliance'] = True   # 默认合规
    
    # 1. 验证subject: 必须是现任董监高
    valid_subject = df['股东身份'] == '董监高'
    df.loc[valid_subject, 'meu_14_3_subject'] = True
    
    # 2. 验证condition: 被立案调查/侦查期间
    investigation_status = [
        '被中国证监会及其派出机构立案调查',
        '被司法机关立案侦查'
    ]
    valid_condition = df['股东涉嫌证券期货违法犯罪事件'].isin(investigation_status)
    df.loc[valid_condition, 'meu_14_3_condition'] = True
    
    # 3. 验证constraint: 不得减持股份
    # 检查当日是否有减持行为(当日减持比例>0)
    has_reduced = df['当日减持比例'] > 0
    df.loc[has_reduced, 'meu_14_3_constraint'] = False
    
    # 判断违规情况: 同时满足主体和条件但违反约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        has_reduced
    )
    df.loc[is_violation, 'meu_14_3_compliance'] = False
    
    return df

---

## MEU_14_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 本人因涉嫌与该上市公司有关的证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1436 |
| completion_tokens | 787 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_4(df):
    '''
    检查MEU_14_4合规性:
    "subject": "上市公司董监高", 
    "condition": "本人因涉嫌与该上市公司有关的证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月", 
    "constraint": "不得减持其所持有的本公司股份", 
    "contextual_info": NaN
    
    实现逻辑:
    1. subject: 检查是否为上市公司董监高
    2. condition: 检查是否因涉嫌证券期货违法犯罪被立案调查/侦查，且在行政处罚/刑事判决后未满6个月
    3. constraint: 检查是否有减持行为
    '''
    
    # 初始化结果列
    df['meu_14_4_subject'] = False
    df['meu_14_4_condition'] = False
    df['meu_14_4_constraint'] = True  # 默认满足约束(没有减持)
    df['meu_14_4_compliance'] = True  # 默认合规
    
    # 1. 检查subject条件: 上市公司董监高
    is_director = df['股东身份'] == '董监高'
    df.loc[is_director, 'meu_14_4_subject'] = True
    
    # 2. 检查condition条件: 被立案调查且在处罚后未满6个月
    # 先分组处理每个股东的时间线
    grouped = df.groupby(['公司简称', '股东'])
    
    # 为每个股东计算condition条件
    for (company, shareholder), group in grouped:
        # 获取该股东的相关事件
        events = group['股东涉嫌证券期货违法犯罪事件'].dropna().unique()
        
        # 检查是否有被立案调查/侦查的事件
        has_investigation = any(e in ['被中国证监会及其派出机构立案调查', '被司法机关立案侦查'] for e in events)
        
        if has_investigation:
            # 获取行政处罚/刑事判决的日期
            penalty_dates = group.loc[
                group['股东涉嫌证券期货违法犯罪事件'].isin(['行政处罚决定作出']), '日期'
            ]
            
            # 如果有行政处罚/刑事判决
            if not penalty_dates.empty:
                latest_penalty_date = penalty_dates.max()
                
                # 计算每个交易日距离处罚日的天数
                days_since_penalty = (group['日期'] - latest_penalty_date).dt.days
                
                # condition满足: 处罚后未满6个月(约180天)
                condition_met = (days_since_penalty >= 0) & (days_since_penalty < 180)
                
                # 标记condition
                condition_indices = group[condition_met].index
                df.loc[condition_indices, 'meu_14_4_condition'] = True
                
                # 3. 检查constraint: 是否有减持行为
                # 减持行为包括: 当日减持比例>0 或 存在减持计划
                has_reduction = (
                    (group['当日减持比例'] > 0) | 
                    (group['存在减持计划'])
                )
                
                # 标记违反constraint的行
                violation_indices = group[condition_met & has_reduction].index
                df.loc[violation_indices, 'meu_14_4_constraint'] = False
                df.loc[violation_indices, 'meu_14_4_compliance'] = False
    
    return df

---

## MEU_14_5


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 本人因涉及证券期货违法，被中国证监会行政处罚，罚没款尚未足额缴纳，且不存在法律、行政法规另有规定或减持资金用于缴纳罚没款的情况 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | 不考虑法律、行政法规另有规定或减持资金用于缴纳罚没款的情况 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1452 |
| completion_tokens | 585 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_5(df):
    '''
    检查MEU_14_5合规性:
    "subject": "上市公司董监高", 
    "condition": "本人因涉及证券期货违法，被中国证监会行政处罚，罚没款尚未足额缴纳，且不存在法律、行政法规另有规定或减持资金用于缴纳罚没款的情况", 
    "constraint": "不得减持其所持有的本公司股份", 
    "contextual_info": NaN
    
    处理逻辑:
    1. subject: 检查是否为董监高(不包括离任董监高)
    2. condition: 
       - 涉及证券期货违法(行政处罚决定作出)
       - 罚没款未足额缴纳(假设只要处于行政处罚决定作出状态即视为未足额缴纳)
       - 不考虑法律行政法规另有规定或减持资金用于缴纳罚没款的情况(根据题目要求)
    3. constraint: 不得减持股份(检查是否有减持行为)
    '''
    
    # 初始化结果列
    df['meu_14_5_subject'] = False
    df['meu_14_5_condition'] = False
    df['meu_14_5_constraint'] = True  # 默认满足约束(没有减持)
    df['meu_14_5_compliance'] = True  # 默认合规
    
    # 1. 检查subject条件: 当前是董监高(不包括离任董监高)
    is_director = df['股东身份'] == '董监高'
    df.loc[is_director, 'meu_14_5_subject'] = True
    
    # 2. 检查condition条件
    # 被行政处罚且未足额缴纳(假设"行政处罚决定作出"状态即表示未足额缴纳)
    has_penalty = df['股东涉嫌证券期货违法犯罪事件'] == '行政处罚决定作出'
    df.loc[has_penalty, 'meu_14_5_condition'] = True
    
    # 3. 检查constraint条件: 不得减持股份
    # 检查是否有减持行为(当日减持比例>0)
    has_reduction = df['当日减持比例'] > 0
    df.loc[has_reduction, 'meu_14_5_constraint'] = False
    
    # 判断违规情况: 同时满足subject和condition但不满足constraint
    is_violation = (
        is_director & 
        has_penalty & 
        has_reduction
    )
    
    # 标记违规行
    df.loc[is_violation, 'meu_14_5_compliance'] = False
    
    return df

---

## MEU_14_6


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 本人因涉及与本上市公司有关的违法违规，被证券交易所公开谴责未满三个月 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1416 |
| completion_tokens | 714 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_6(df):
    '''
    检查上市公司董监高在被交易所公开谴责后三个月内是否违规减持股份
    
    法律的最小可执行单元内容:
    "subject": "上市公司董监高", 
    "condition": "本人因涉及与本上市公司有关的违法违规，被证券交易所公开谴责未满三个月", 
    "constraint": "不得减持其所持有的本公司股份", 
    "contextual_info": NaN
    
    实现逻辑:
    1. 验证主体: 必须是上市公司董监高
    2. 验证条件: 该股东因涉及与本公司有关的违法违规被交易所公开谴责且未满三个月
    3. 验证约束: 在谴责后三个月内不得减持股份
    '''
    
    # 初始化结果列
    df['meu_14_6_subject'] = False
    df['meu_14_6_condition'] = False
    df['meu_14_6_constraint'] = True  # 默认满足约束(未减持)
    df['meu_14_6_compliance'] = True  # 默认合规
    
    # 1. 验证主体条件 - 必须是上市公司董监高
    is_director = df['股东身份'] == '董监高'
    df.loc[is_director, 'meu_14_6_subject'] = True
    
    # 2. 验证条件 - 被交易所公开谴责未满三个月
    # 先找出所有被公开谴责的记录
    condemned = df[df['股东涉嫌证券期货违法犯罪事件'] == '被本所公开谴责'].copy()
    
    # 如果存在被公开谴责的记录
    if not condemned.empty:
        # 计算每个被谴责股东的三个月期限
        condemned['三个月期限'] = condemned['日期'] + pd.DateOffset(months=3)
        
        # 为每个被谴责股东创建时间范围
        for _, row in condemned.iterrows():
            company = row['公司简称']
            shareholder = row['股东']
            start_date = row['日期']
            end_date = row['三个月期限']
            
            # 标记该股东在谴责后三个月内的所有记录
            mask = (
                (df['公司简称'] == company) & 
                (df['股东'] == shareholder) & 
                (df['日期'] >= start_date) & 
                (df['日期'] <= end_date)
            )
            df.loc[mask, 'meu_14_6_condition'] = True
    
    # 3. 验证约束 - 在谴责后三个月内不得减持
    # 减持行为表现为: 当日减持比例 > 0 或 存在减持计划为True
    is_reducing = (df['当日减持比例'] > 0) | (df['存在减持计划'])
    
    # 标记违规减持行为: 满足主体和条件但有减持行为
    violation_mask = (
        df['meu_14_6_subject'] & 
        df['meu_14_6_condition'] & 
        is_reducing
    )
    df.loc[violation_mask, 'meu_14_6_constraint'] = False
    df.loc[violation_mask, 'meu_14_6_compliance'] = False
    
    return df

---

## MEU_14_7


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司股票因可能触及重大违法强制退市情形而被本所实施退市风险警示，且处于本所规定的限制转让的期限内 |
| constraint | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1430 |
| completion_tokens | 785 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_7(df):
    '''
    检查MEU_14_7合规性:
    "subject": "上市公司董监高", 
    "condition": "上市公司股票因可能触及重大违法强制退市情形而被本所实施退市风险警示，且处于本所规定的限制转让的期限内", 
    "constraint": "不得减持其所持有的本公司股份", 
    "contextual_info": NaN
    
    实现逻辑:
    1. subject: 检查是否为董监高
    2. condition: 检查公司是否被实施退市风险警示且处于限制转让期
    3. constraint: 检查是否有减持行为
    4. 违规情况: 满足subject和condition但不满足constraint
    '''
    
    # 初始化结果列
    df['meu_14_7_subject'] = False
    df['meu_14_7_condition'] = False
    df['meu_14_7_constraint'] = True  # 默认满足约束(没有减持)
    df['meu_14_7_compliance'] = True  # 默认合规
    
    # 1. 检查subject: 上市公司董监高
    is_director = df['股东身份'] == '董监高'
    df.loc[is_director, 'meu_14_7_subject'] = True
    
    # 2. 检查condition: 公司被实施退市风险警示且处于限制转让期
    # 假设退市风险警示和限制转让期信息可以从公司涉嫌证券期货违法犯罪事件中判断
    is_risk_warning = df['公司涉嫌证券期货违法犯罪事件'] == '被中国证监会及其派出机构立案调查'
    # 假设限制转让期为立案调查开始到调查结束或行政处罚决定作出
    # 需要按公司分组计算限制转让期
    def get_restriction_period(group):
        investigation_start = group[group['公司涉嫌证券期货违法犯罪事件'] == '被中国证监会及其派出机构立案调查']['日期']
        investigation_end = group[group['公司涉嫌证券期货违法犯罪事件'].isin([
            '中国证监会及其派出机构立案调查结束', 
            '行政处罚决定作出'
        ])]['日期']
        
        if not investigation_start.empty:
            start_date = investigation_start.min()
            end_date = investigation_end.min() if not investigation_end.empty else pd.Timestamp.max
            group['in_restriction'] = group['日期'].between(start_date, end_date)
            return group
        else:
            group['in_restriction'] = False
            return group
    
    df = df.groupby('公司简称', group_keys=False).apply(get_restriction_period)
    valid_condition = is_risk_warning & df['in_restriction']
    df.loc[valid_condition, 'meu_14_7_condition'] = True
    
    # 3. 检查constraint: 不得减持
    # 检查是否有减持行为(当日减持比例>0)
    has_reduction = df['当日减持比例'] > 0
    df.loc[has_reduction, 'meu_14_7_constraint'] = False
    
    # 4. 判断违规情况: 满足subject和condition但不满足constraint
    is_violation = is_director & valid_condition & has_reduction
    df.loc[is_violation, 'meu_14_7_compliance'] = False
    
    # 清理临时列
    df.drop('in_restriction', axis=1, inplace=True, errors='ignore')
    
    return df

---

# Law Article 16

## MEU_16_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司年度报告、半年度报告公告前15日内 |
| constraint | 不得买卖本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1410 |
| completion_tokens | 764 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_16_1(df):
    '''
    检查上市公司董监高在定期报告公告前15日内是否买卖本公司股份
    
    法律的最小可执行单元内容:
    {"subject": "上市公司董监高", 
     "condition": "上市公司年度报告、半年度报告公告前15日内", 
     "constraint": "不得买卖本公司股份", 
     "contextual_info": NaN}
    
    检查逻辑:
    1. subject: 股东身份为董监高
    2. condition: 当前日期在年度/半年度报告公告前15日内
    3. constraint: 不得买卖本公司股份(通过日收益率判断是否有交易)
    '''
    
    # 初始化结果列
    df['meu_16_1_subject'] = False
    df['meu_16_1_condition'] = False
    df['meu_16_1_constraint'] = True  # 默认满足约束(没有交易)
    df['meu_16_1_compliance'] = True  # 默认合规
    
    # 1. 检查subject条件: 股东身份为董监高
    is_director = df['股东身份'] == '董监高'
    df.loc[is_director, 'meu_16_1_subject'] = True
    
    # 2. 检查condition条件: 在定期报告公告前15日内
    # 获取年度/半年度报告公告日期
    report_announcements = df[df['公告类型'].isin(['年度报告', '半年度报告'])]
    
    # 为每个公司创建公告日期范围(公告日前15日内)
    announcement_ranges = []
    for _, group in report_announcements.groupby(['公司简称', '公告类型']):
        # 取最早的公告日期(防止多次公告)
        min_date = group['公告日期'].min()
        start_date = min_date - pd.Timedelta(days=15)
        announcement_ranges.append({
            '公司简称': group['公司简称'].iloc[0],
            '公告类型': group['公告类型'].iloc[0],
            'start_date': start_date,
            'end_date': min_date
        })
    
    # 转换为DataFrame
    announcement_ranges_df = pd.DataFrame(announcement_ranges)
    
    # 标记处于公告前15日内的记录
    for _, row in announcement_ranges_df.iterrows():
        mask = (
            (df['公司简称'] == row['公司简称']) &
            (df['日期'] >= row['start_date']) & 
            (df['日期'] < row['end_date'])
        )
        df.loc[mask, 'meu_16_1_condition'] = True
    
    # 3. 检查constraint条件: 不得买卖本公司股份
    # 假设日收益率不为0表示有交易(简化处理)
    has_trade = df['日收益率'] != 0
    df.loc[has_trade, 'meu_16_1_constraint'] = False
    
    # 综合判断违规情况: 满足主体和条件但不满足约束
    is_violation = (
        (df['meu_16_1_subject']) & 
        (df['meu_16_1_condition']) & 
        (~df['meu_16_1_constraint'])
    )
    
    df.loc[is_violation, 'meu_16_1_compliance'] = False
    
    return df

---

## MEU_16_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司季度报告、业绩预告、业绩快报公告前5日内 |
| constraint | 不得买卖本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1413 |
| completion_tokens | 669 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_16_2(df):
    '''
    检查上市公司董监高在季度报告、业绩预告、业绩快报公告前5日内是否买卖本公司股份
    
    "subject": "上市公司董监高", 
    "condition": "上市公司季度报告、业绩预告、业绩快报公告前5日内", 
    "constraint": "不得买卖本公司股份", 
    "contextual_info": NaN
    
    实现逻辑:
    1. 检查主体: 股东身份为"董监高"
    2. 检查条件: 日期在公告前5日内(自然日)
    3. 检查约束: 不得有交易行为(通过日收益率判断)
    '''
    
    # 初始化结果列
    df['meu_16_2_subject'] = False
    df['meu_16_2_condition'] = False
    df['meu_16_2_constraint'] = True  # 默认满足约束(没有交易)
    df['meu_16_2_compliance'] = True  # 默认合规
    
    # 1. 检查主体条件: 股东身份为"董监高"
    is_director = df['股东身份'] == '董监高'
    df.loc[is_director, 'meu_16_2_subject'] = True
    
    # 2. 检查时间条件: 在公告前5日内
    # 获取所有相关公告的日期
    relevant_announcements = df[df['公告类型'].isin(['季度报告', '业绩预告', '业绩快报'])]
    
    # 为每个公司创建公告日期列表
    announcement_dates = relevant_announcements.groupby('公司简称')['公告日期'].unique()
    
    # 检查每个日期是否在任意公告前5日内
    def is_in_restricted_period(row):
        company = row['公司简称']
        if company in announcement_dates:
            for ann_date in announcement_dates[company]:
                if (ann_date - row['日期']).days <= 5 and (ann_date - row['日期']).days >= 0:
                    return True
        return False
    
    df['meu_16_2_condition'] = df.apply(is_in_restricted_period, axis=1)
    
    # 3. 检查约束条件: 不得有交易行为
    # 通过日收益率判断是否有交易(非零表示有交易)
    has_trade = df['日收益率'] != 0
    df.loc[has_trade, 'meu_16_2_constraint'] = False
    
    # 判断违规情况: 同时满足主体和条件但不满足约束
    is_violation = (
        df['meu_16_2_subject'] & 
        df['meu_16_2_condition'] & 
        (~df['meu_16_2_constraint'])
    )
    
    df.loc[is_violation, 'meu_16_2_compliance'] = False
    
    return df

---

# Law Article 17

## MEU_17_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 在其就任时确定的任期内和任期届满后6个月内，每年通过集中竞价、大宗交易、协议转让等方式转让股份且不属于因司法强制执行、继承、遗赠、依法分割财产等导致股份变动 |
| constraint | 每年转让的股份不得超过其所持本公司股份总数的25% |
| contextual_info | nan |
| note | nan |
| relation | depend |
| target | MEU_18_1 |
| type | 实际执行单元 |
| comments | 不考虑因司法强制执行、继承、遗赠、依法分割财产等导致股份变动的情况 |
| prompt_tokens | 1452 |
| completion_tokens | 941 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_17_1(df):
    '''
    检查MEU_17_1合规性:
    "subject": "上市公司董监高", 
    "condition": "在其就任时确定的任期内和任期届满后6个月内，每年通过集中竞价、大宗交易、协议转让等方式转让股份且不属于因司法强制执行、继承、遗赠、依法分割财产等导致股份变动", 
    "constraint": "每年转让的股份不得超过其所持本公司股份总数的25%", 
    "contextual_info": NaN
    
    实现逻辑:
    1. subject: 检查是否为董监高或离任董监高(在离任后6个月内)
    2. condition: 
       - 检查减持方式是否为集中竞价、大宗交易、协议转让
       - 排除司法强制执行、继承等特殊原因导致的股份变动
    3. constraint: 计算每年减持比例是否超过25%
    '''
    
    # 初始化结果列
    df['meu_17_1_subject'] = False
    df['meu_17_1_condition'] = False
    df['meu_17_1_constraint'] = False
    df['meu_17_1_compliance'] = True  # 默认合规
    
    # 1. 检查subject条件: 董监高或离任6个月内的董监高
    is_director = df['股东身份'] == '董监高'
    
    # 计算离任后6个月内的情况
    is_former_director = df['股东身份'] == '离任董监高'
    within_6months = (df['日期'] - df['离任日期']).dt.days <= 180
    valid_former_director = is_former_director & within_6months
    
    valid_subject = is_director | valid_former_director
    df.loc[valid_subject, 'meu_17_1_subject'] = True
    
    # 2. 检查condition条件
    # 有效的减持方式
    valid_methods = ['竞价交易', '大宗交易', '协议转让']
    valid_method = df['减持方式'].isin(valid_methods)
    
    # 排除特殊原因的股份变动(假设这些情况会有特殊标记)
    # 这里假设如果减持方式为空且持股数量减少，则可能是特殊原因
    # 实际情况可能需要更精确的字段判断
    not_special_case = ~df['减持方式'].isna()
    
    valid_condition = valid_method & not_special_case
    df.loc[valid_condition, 'meu_17_1_condition'] = True
    
    # 3. 检查constraint条件: 每年减持不超过25%
    # 需要按公司和股东分组，计算每年的减持比例
    
    # 先筛选出满足subject和condition的记录
    target_records = df[valid_subject & valid_condition].copy()
    
    if not target_records.empty:
        # 按公司和股东分组，计算每年减持比例
        grouped = target_records.groupby(['公司简称', '股东', target_records['日期'].dt.year])
        
        # 计算每组的累计减持比例
        cum_annual_reduction = grouped['当日减持比例'].cumsum()
        
        # 检查是否超过25%
        valid_constraint = cum_annual_reduction <= 0.25
        
        # 将结果映射回原DataFrame
        # 由于groupby操作改变了索引，我们需要先重置索引
        target_records.reset_index(inplace=True)
        target_records['meu_17_1_constraint'] = valid_constraint.values
        
        # 更新原DataFrame
        for idx, row in target_records.iterrows():
            orig_idx = row['index']  # 保存的原索引
            df.loc[orig_idx, 'meu_17_1_constraint'] = row['meu_17_1_constraint']
    
    # 标记违规情况: 满足subject和condition但不满足constraint
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~df['meu_17_1_constraint'])
    )
    df.loc[is_violation, 'meu_17_1_compliance'] = False
    
    return df

---

## MEU_17_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 所持股份不超过1000股 |
| constraint | 可一次全部转让且不受前款转让比例限制 |
| contextual_info | nan |
| note | nan |
| relation | exclude |
| target | MEU_17_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1413 |
| completion_tokens | 469 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_17_2(df):
    '''
    检查MEU_17_2合规性:
    "subject": "上市公司董监高", 
    "condition": "所持股份不超过1000股", 
    "constraint": "可一次全部转让且不受前款转让比例限制", 
    "contextual_info": NaN
    
    合规条件:
    1. 主体必须是上市公司董监高(不包括离任董监高)
    2. 持股数量不超过1000股
    3. 可以一次性全部转让(不受其他减持比例限制)
    '''
    
    # 初始化结果列
    df['meu_17_2_subject'] = False
    df['meu_17_2_condition'] = False
    df['meu_17_2_constraint'] = False
    df['meu_17_2_compliance'] = True  # 默认合规
    
    # 1. 验证subject: 必须是上市公司董监高(不包括离任的)
    is_director = df['股东身份'] == '董监高'
    
    # 2. 验证condition: 持股数量不超过1000股
    share_limit = df['持股数量'] <= 1000
    
    # 3. 验证constraint: 可以一次性全部转让(不受其他限制)
    # 对于该MEU来说，只要满足主体和条件，就自动满足约束条件
    # 因为约束就是允许一次性转让
    
    # 标记各条件满足情况
    df.loc[is_director, 'meu_17_2_subject'] = True
    df.loc[share_limit, 'meu_17_2_condition'] = True
    df.loc[is_director & share_limit, 'meu_17_2_constraint'] = True
    
    # 判断违规情况: 当主体是董监高且持股>1000股时，需要受其他减持限制
    # 但该MEU本身不涉及违规判断，只是标识哪些情况适用该豁免条款
    # 因此不需要标记违规
    
    return df

---

# Law Article 18

## MEU_18_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | nan |
| constraint | nan |
| contextual_info | 以上年末其所持股总数作为基数计算可转让股份数量 |
| note | nan |
| relation | combine |
| target | MEU_18_2;MEU_18_3;MEU_18_4;MEU_18_5;MEU_18_6 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1407 |
| completion_tokens | 637 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_18_1(df):
    '''
    检查MEU_18_1合规性:
    "subject": "上市公司董监高", 
    "contextual_info": "以上年末其所持股总数作为基数计算可转让股份数量"
    
    该条款要求对上市公司董监高的可转让股份数量进行限制，计算基数为上年末持股总数。
    需要验证:
    1. 主体是否为上市公司董监高
    2. 计算可转让股份数量是否基于上年末持股总数
    '''
    
    # 初始化结果列
    df['meu_18_1_subject'] = False
    df['meu_18_1_compliance'] = True  # 默认合规
    
    # 1. 验证subject是否为上市公司董监高
    is_director = df['股东身份'] == '董监高'
    df.loc[is_director, 'meu_18_1_subject'] = True
    
    # 对每个董监高股东，按公司和股东分组处理
    grouped = df[is_director].groupby(['公司简称', '股东'])
    
    for (company, shareholder), group in grouped:
        # 获取该股东上年末持股数量
        # 首先找到每个年份
        years = group['日期'].dt.year.unique()
        
        for year in sorted(years[1:]):  # 从第二年开始检查(需要上年数据)
            prev_year = year - 1
            # 获取上年末(12月31日)的持股数量
            prev_year_end = group[group['日期'].dt.year == prev_year]
            if len(prev_year_end) == 0:
                continue  # 无上年数据，无法验证
                
            # 取上年最后一天的持股数量
            prev_year_end_last = prev_year_end.sort_values('日期').iloc[-1]
            base_quantity = prev_year_end_last['持股数量']
            
            # 获取本年该股东的所有交易
            current_year = group[group['日期'].dt.year == year]
            
            # 计算本年累计减持数量
            current_year['累计减持数量'] = current_year['当日减持比例'] * current_year['总股本']
            total_reduced = current_year['累计减持数量'].sum()
            
            # 检查是否超过可转让数量(假设可转让数量为上年末持股的25%)
            max_allowed = base_quantity * 0.25
            if total_reduced > max_allowed:
                # 标记违规
                violation_dates = current_year[current_year['累计减持数量'].cumsum() > max_allowed]['日期']
                df.loc[(df['公司简称'] == company) & 
                       (df['股东'] == shareholder) & 
                       (df['日期'].isin(violation_dates)), 
                       'meu_18_1_compliance'] = False
    
    return df

---

## MEU_18_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | nan |
| constraint | nan |
| contextual_info | 当年可转让但未转让的本公司股份，应当计入当年末其所持有本公司股份的总数，以该总数作为次年可转让股份的计算基数 |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1426 |
| completion_tokens | 647 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_18_2(df):
    '''
    检查MEU_18_2合规性：
    "subject": "上市公司董监高", 
    "contextual_info": "当年可转让但未转让的本公司股份，应当计入当年末其所持有本公司股份的总数，以该总数作为次年可转让股份的计算基数"
    
    实现逻辑：
    1. 筛选出董监高股东
    2. 按年份分组计算每年末持股数量
    3. 检查次年可转让股份计算基数是否正确
    '''
    
    # 初始化结果列
    df['meu_18_2_compliance'] = True
    df['meu_18_2_violation_reason'] = ''
    
    # 1. 筛选董监高股东 (subject条件)
    is_director = df['股东身份'].isin(['董监高', '离任董监高'])
    directors_df = df[is_director].copy()
    
    if directors_df.empty:
        return df
    
    # 2. 按公司和股东分组处理
    grouped = directors_df.groupby(['公司简称', '股东'])
    
    for (company, shareholder), group in grouped:
        # 获取该股东每年的年末持股情况
        group['year'] = group['日期'].dt.year
        
        # 找出每年最后交易日的记录
        year_end_records = group.sort_values('日期').groupby('year').last()
        
        # 计算次年可转让股份基数
        for i in range(len(year_end_records)-1):
            current_year = year_end_records.index[i]
            next_year = current_year + 1
            
            # 获取当前年末持股数量
            current_year_end_holdings = year_end_records.loc[current_year, '持股数量']
            
            # 检查次年记录中的持股基数是否正确
            next_year_records = group[group['year'] == next_year]
            
            if not next_year_records.empty:
                # 获取次年首次记录中的持股基数
                first_next_year_record = next_year_records.sort_values('日期').iloc[0]
                
                # 检查是否使用了正确的计算基数
                if first_next_year_record['持股数量'] != current_year_end_holdings:
                    # 标记违规记录
                    violation_dates = next_year_records['日期']
                    df.loc[(df['公司简称'] == company) & 
                          (df['股东'] == shareholder) & 
                          (df['日期'].isin(violation_dates)),
                          'meu_18_2_compliance'] = False
                    
                    df.loc[(df['公司简称'] == company) & 
                          (df['股东'] == shareholder) & 
                          (df['日期'].isin(violation_dates)),
                          'meu_18_2_violation_reason'] = f'次年可转让股份计算基数错误: 应为{current_year_end_holdings}'
    
    return df

---

## MEU_18_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | nan |
| constraint | nan |
| contextual_info | 所持本公司股份在年内增加的，新增无限售条件股份当年可转让25%，新增有限售条件股份计入次年可转让股份的计算基数。 |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1426 |
| completion_tokens | 594 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_18_3(df):
    '''
    检查MEU_18_3合规性:
    "subject": "上市公司董监高", 
    "contextual_info": "所持本公司股份在年内增加的，新增无限售条件股份当年可转让25%，新增有限售条件股份计入次年可转让股份的计算基数。"
    
    实现逻辑:
    1. 首先筛选出董监高股东
    2. 计算每个股东年内股份增加情况
    3. 区分无限售条件和有限售条件股份
    4. 检查转让比例是否符合规定
    '''
    
    # 初始化结果列
    df['meu_18_3_subject'] = False
    df['meu_18_3_compliance'] = True  # 默认合规
    
    # 1. 筛选符合条件的subject: 上市公司董监高
    is_director = df['股东身份'] == '董监高'
    df.loc[is_director, 'meu_18_3_subject'] = True
    
    # 对每个公司的每个董监高股东单独处理
    for (company, shareholder), group in df[is_director].groupby(['公司简称', '股东']):
        # 按日期排序
        group = group.sort_values('日期')
        
        # 计算持股数量变化
        group['持股变化'] = group['持股数量'].diff().fillna(0)
        
        # 标记年内数据 (同一年份)
        group['年份'] = group['日期'].dt.year
        for year, year_group in group.groupby('年份'):
            # 计算年内总增加量
            year_increase = year_group[year_group['持股变化'] > 0]['持股变化'].sum()
            
            if year_increase > 0:
                # 这里简化处理: 假设所有新增股份都是无限售条件股份
                # 实际应用中需要根据股份性质区分有限售和无限售条件股份
                max_allowed_sell = year_increase * 0.25
                
                # 计算年内实际减持量
                year_sell = year_group[year_group['持股变化'] < 0]['持股变化'].sum() * -1
                
                # 检查是否超额减持
                if year_sell > max_allowed_sell:
                    # 标记违规日期
                    violation_dates = year_group[year_group['持股变化'] < 0]['日期']
                    df.loc[(df['公司简称'] == company) & 
                          (df['股东'] == shareholder) & 
                          (df['日期'].isin(violation_dates)),
                          'meu_18_3_compliance'] = False
    
    return df

---

## MEU_18_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | nan |
| constraint | nan |
| contextual_info | 因上市公司实施权益分派导致所持本公司股份增加的，可同比例增加当年可转让数量 |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1416 |
| completion_tokens | 534 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_18_4(df):
    '''
    检查上市公司董监高因权益分派导致持股增加时，是否可同比例增加当年可转让数量
    
    法律的最小可执行单元内容: 
    {
        "subject": "上市公司董监高", 
        "condition": NaN, 
        "constraint": NaN, 
        "contextual_info": "因上市公司实施权益分派导致所持本公司股份增加的，可同比例增加当年可转让数量"
    }
    
    处理逻辑:
    1. 首先筛选出主体为上市公司董监高的记录
    2. 检查这些记录是否因权益分派导致持股数量增加
    3. 如果满足条件，则标记为合规
    '''
    
    # 初始化结果列
    df['meu_18_4_subject'] = False
    df['meu_18_4_compliance'] = True  # 默认合规
    
    # 1. 验证subject条件：上市公司董监高
    is_director = df['股东身份'] == '董监高'
    df.loc[is_director, 'meu_18_4_subject'] = True
    
    # 2. 检查权益分派导致的持股增加情况
    # 按公司和股东分组，计算持股数量的变化
    grouped = df[is_director].groupby(['公司简称', '股东'])
    
    # 初始化权益分派标记列
    df['is_equity_distribution'] = False
    
    for (company, shareholder), group in grouped:
        # 按日期排序
        group = group.sort_values('日期')
        
        # 计算持股数量的变化
        group['持股变化'] = group['持股数量'].diff()
        
        # 检查是否有权益分派公告
        has_equity_announcement = group['公告类型'].str.contains('权益分派', na=False)
        
        # 标记因权益分派导致持股增加的记录
        equity_increase = (group['持股变化'] > 0) & has_equity_announcement
        indices = group[equity_increase].index
        df.loc[indices, 'is_equity_distribution'] = True
        
        # 对于这些记录，标记为合规（可以同比例增加当年可转让数量）
        df.loc[indices, 'meu_18_4_compliance'] = True
    
    return df

---

# Law Article 23

## MEU_23_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 | 董监高 |
| condition | nan |
| constraint | 不得融券卖出本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1405 |
| completion_tokens | 510 |


### 推理内容

reasoning content not found

### 代码实现

In [ ]:
import pandas as pd

def check_meu_23_1(df):
    '''
    检查上市公司大股东或董监高是否违规融券卖出本公司股份
    
    法律的最小可执行单元内容:
    {"subject": "上市公司大股东 | 董监高", 
     "condition": NaN, 
     "constraint": "不得融券卖出本公司股份", 
     "contextual_info": NaN}
     
    合规性检查逻辑:
    1. 主体(subject): 必须是上市公司大股东或董监高
    2. 约束(constraint): 不得进行融券卖出操作
    '''
    
    # 初始化结果列
    df['meu_23_1_subject'] = False  # 主体是否符合
    df['meu_23_1_violation'] = False  # 是否违规
    
    # 1. 检查主体条件: 上市公司大股东或董监高
    # 大股东定义: 控股股东、实际控制人或持股5%以上股东
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人']) |
        (df['持股比例'] >= 0.05)  # 包括未明确标注但持股5%以上的股东
    )
    
    # 董监高定义: 包括现任和离任董监高
    is_director = df['股东身份'].isin(['董监高', '离任董监高'])
    
    # 合并主体条件
    valid_subject = is_major_shareholder | is_director
    
    # 2. 检查约束条件: 是否进行了融券卖出
    # 融券卖出定义: 减持方式为'融券卖出'
    is_short_selling = df['减持方式'] == '融券卖出'
    
    # 标记主体符合情况
    df.loc[valid_subject, 'meu_23_1_subject'] = True
    
    # 标记违规情况: 主体符合且进行了融券卖出
    violation = valid_subject & is_short_selling
    df.loc[violation, 'meu_23_1_violation'] = True
    
    # 添加合规性总评列
    df['meu_23_1_compliance'] = ~violation
    
    return df

---